In [1]:
print("Installing RDKit, Mordred, and Pandas...")
!pip install rdkit mordred pandas -q
print("Installation complete.")

Installing RDKit, Mordred, and Pandas...
Installation complete.


In [2]:
import pandas as pd
from rdkit import Chem
from rdkit.Chem import Descriptors, Lipinski, Crippen, MolSurf  # Import specific descriptor modules
from mordred import Calculator, descriptors                      # Mordred calculator and descriptor list
import numpy as np
import warnings

# Suppress RDKit and Mordred warnings for cleaner output (optional)
warnings.filterwarnings("ignore", category=UserWarning)
# from rdkit import rdBase
# rdBase.DisableLog('rdApp.warning') # More aggressive RDKit warning suppression if needed

print("Libraries imported.")

Libraries imported.


In [3]:
csv_content = """Smiles
COCCCCC(=NOCCN)C1=CC=C(C=C1)C(F)(F)F
CCCC(C)(COC(=O)N)COC(=O)N
CN(C)CCC=C1C2=CC=CC=C2CCC3=CC=CC=C31
[Li+].[Li+].[Li+].C(C(=O)[O-])C(CC(=O)[O-])(C(=O)[O-])O
CN1CCN(CC1)C2=NC3=C(C=CC(=C3)Cl)NC4=CC=CC=C42
C1=CC=C(C(=C1)C2=NC(C(=O)NC3=C2C=C(C=C3)Cl)O)Cl
C1C2=NN=CN2C3=C(C=C(C=C3)Cl)C(=N1)C4=CC=CC=C4
CC(C)CC(CC(=O)O)CN
CN=C1CN(C(=C2C=C(C=CC2=N1)Cl)C3=CC=CC=C3)O
CC1=NN=C2N1C3=C(C=C(C=C3)Cl)C(=NC2)C4=CC=CC=C4
CN1CCN(CC1)C2=NC3=CC=CC=C3OC4=C2C=C(C=C4)Cl
CCN(CC)CCN1C(=O)CN=C(C2=C1C=CC(=C2)Cl)C3=CC=CC=C3F
CNCCCC1C2=CC=CC=C2C=CC3=CC=CC=C13
C1CN(CCN1CCCCC2=CNC3=C2C=C(C=C3)C#N)C4=CC5=C(C=C4)OC(=C5)C(=O)N
CCN(CC)C(=O)C1(CC1CN)C2=CC=CC=C2
CCCCCCCCCCCC(=O)OCN1C(=O)CCC2=C1C=C(C=C2)OCCCCN3CCN(CC3)C4=C(C(=CC=C4)Cl)Cl
CCCCCCC(C)(C)C1=CC(=C2C3CC(=O)CCC3C(OC2=C1)(C)C)O
C1CCC(C(C1)CN2CCN(CC2)C3=NSC4=CC=CC=C43)CN5C(=O)C6C7CCC(C7)C6C5=O
CN1C(=O)CC(=O)N(C2=C1C=CC(=C2)Cl)C3=CC=CC=C3
CN(C)CCCC1(C2=C(CO1)C=C(C=C2)C#N)C3=CC=C(C=C3)F
C1CNCC(C1C2=CC=C(C=C2)F)COC3=CC4=C(C=C3)OCO4
C1CN(CCN1CCC2=C(C=C3C(=C2)CC(=O)N3)Cl)C4=NSC5=CC=CC=C54
CC1=CC2=C(S1)NC3=CC=CC=C3N=C2N4CCN(CC4)C
C1CN(CCC1(C2=CC=C(C=C2)Cl)O)CCCC(=O)C3=CC=C(C=C3)F
C1CN(CCN1)C2=NC3=CC=CC=C3OC4=C2C=C(C=C4)Cl
C1CN(CCN1CCCN2C(=O)N3C=CC=CC3=N2)C4=CC(=CC=C4)Cl
CCOC1=CC=CC=C1OCC2CNCCO2
CC(CN1C2=CC=CC=C2CCC3=CC=CC=C31)CN(C)C
CNCCCN1C2=CC=CC=C2CCC3=CC=CC=C31
CCCC(CCC)C(=O)O
CNCCC=C1C2=CC=CC=C2CCC3=CC=CC=C31
CN(C)CCCN1C2=CC=CC=C2CCC3=CC=CC=C31
CC1C(OCCN1C)C2=CC=CC=C2
CC1=NN=C2N1C3=C(C=C(C=C3)Cl)C(=NC2)C4=CC=CC=C4Cl
CCCC(C)(COC(=O)N)COC(=O)NC(C)C
CC1(CCC2C(C1)CCC3C2CCC4(C3CCC4C(=O)CN5C=C(C=N5)C#N)C)O
C1=CC=C2C(=C1)C(=CN2)CC(C(=O)O)N
CC1(CC(=O)N(C(=O)C1)CCCCN2CCN(CC2)C3=NC=CC=N3)C
C1=CC=C(C=C1)C2=NC(C(=O)NC3=C2C=C(C=C3)Cl)O
C1CCC(CC1)(CC(=O)O)CN
CCC1=C(NC2=C1C(=O)C(CC2)CN3CCOCC3)C
[Li+].[Li+].C(=O)([O-])[O-]
C1CN(CC=C1N2C3=CC=CC=C3NC2=O)CCCC(=O)C4=CC=C(C=C4)F
C1=CC(=C(C(=C1)Cl)Cl)C2=C(N=C(N=N2)N)N
CCN1CCCC1CNC(=O)C2=CC(=C(C=C2OC)N)S(=O)(=O)CC
CC1=NC=C2N1C3=C(C=C(C=C3)Cl)C(=NC2)C4=CC=CC=C4F
CNC1(CCCCC1=O)C2=CC=CC=C2Cl
C1CN(CCN1CCCN2C3=CC=CC=C3SC4=C2C=C(C=C4)C(F)(F)F)CCO
C1CN(CCN1CCCN2C3=CC=CC=C3SC4=C2C=C(C=C4)Cl)CCO
CCN(CC)C(=O)C1(CC1CN)C2=CC=CC=C2
CC(C(=O)C1=CC(=CC=C1)Cl)NC(C)(C)C
CC1=CC(=C(C=C1)SC2=CC=CC=C2N3CCNCC3)C
CN(C)CC(C1=CC=C(C=C1)OC)C2(CCCCC2)O
CN1CCCCC1CCN2C3=CC=CC=C3SC4=C2C=C(C=C4)SC
CC1=C(C(=O)N2CCCC(C2=N1)O)CCN3CCC(CC3)C4=NOC5=C4C=CC(=C5)F
C1=CC=C(C=C1)CCNN
CC1=C(C(=O)N2CCCCC2=N1)CCN3CCC(CC3)C4=NOC5=C4C=CC(=C5)F
C1CN(CCN1CCOCCO)C2=NC3=CC=CC=C3SC4=CC=CC=C42
CN1CC2C(C1)C3=C(C=CC(=C3)Cl)OC4=CC=CC=C24
CC(CC1=CC=CC=C1)N(C)CC#C
C1CN(CCC1N2C3=CC=CC=C3NC2=O)CCCC(C4=CC=C(C=C4)F)C5=CC=C(C=C5)F
CNC1CCC(C2=CC=CC=C12)C3=CC(=C(C=C3)Cl)Cl
CCC1=NN(C(=O)N1CCOC2=CC=CC=C2)CCCN3CCN(CC3)C4=CC(=CC=C4)Cl
CCCCCC1=CC(=C2C3C=C(CCC3C(OC2=C1)(C)C)C)O
CN(C)CCCN1C2=CC=CC=C2CCC3=C1C=C(C=C3)Cl
CN(C)CCCN1C2=CC=CC=C2SC3=C1C=C(C=C3)Cl
CN(C)CCC=C1C2=CC=CC=C2COC3=CC=CC=C31
CN1CCN2C(C1)C3=CC=CC=C3CC4=C2N=CC=C4
C1=CC=C(C=C1)C2=NC(C(=O)NC3=C2C=C(C=C3)Cl)C(=O)O
CN1C2=C(C=C(C=C2)Cl)C(=NC(C1=O)O)C3=CC=CC=C3
CC1=CC(=NO1)C(=O)NNCC2=CC=CC=C2
CN1CCN(CC1)CCCN2C3=CC=CC=C3SC4=C2C=C(C=C4)Cl
CN1CCN(CC1)CCCN2C3=CC=CC=C3SC4=C2C=C(C=C4)C(F)(F)F
CN1CCN(CC1)CCC=C2C3=CC=CC=C3SC4=C2C=C(C=C4)S(=O)(=O)N(C)C
CNCCCC12CCC(C3=CC=CC=C31)C4=CC=CC=C24
CC(=O)C1=CC(=C(C=C1)OCCCN2CCC(CC2)C3=NOC4=C3C=CC(=C4)F)OC
CC1=NC=CN1CC2CCC3=C(C2=O)C4=CC=CC=C4N3C
CN(C)CCCC1(C2=C(CO1)C=C(C=C2)C#N)C3=CC=C(C=C3)F
COC1C(CC2CN3CCC4=C(C3CC2C1C(=O)OC)NC5=C4C=CC(=C5)OC)OC(=O)C6=CC(=C(C(=C6)OC)OC)OC
C1C(C1N)C2=CC=CC=C2
CN(C)CC(C1=CC=C(C=C1)O)C2(CCCCC2)O
CN1C(=O)CN=C(C2=C1C=CC(=C2)Cl)C3=CC=CC=C3
C1CC(=O)NC2=C1C=CC(=C2)OCCCCN3CCN(CC3)C4=C(C(=CC=C4)Cl)Cl
CNCCC(C1=CC=CS1)OC2=CC=CC3=CC=CC=C32
CNCCC(C1=CC=CC=C1)OC2=CC=C(C=C2)C(F)(F)F
CN(C)C(=O)NC1CCC(CC1)CCN2CCN(CC2)C3=C(C(=CC=C3)Cl)Cl
C1=CC=C2C(=C1)C=CC3=CC=CC=C3N2C(=O)N
C1CC(=O)NC(=O)C1N2C(=O)C3=C(C2=O)C(=CC=C3)N
CC(=C(C#N)C(=O)NC1=CC=C(C=C1)C(F)(F)F)O
CCC1C(C(C(N(CC(CC(C(C(C(C(C(=O)O1)C)OC2CC(C(C(O2)C)O)(C)OC)C)OC3C(C(CC(O3)C)N(C)C)O)(C)O)C)C)C)O)(C)O
CC(C)CN1C=NC2=C1C3=CC=CC=C3N=C2N
C1=CC=NC(=C1)NS(=O)(=O)C2=CC=C(C=C2)N=NC3=CC(=C(C=C3)O)C(=O)O
C1CC(=O)NC(=O)C1N2C(=O)C3=CC=CC=C3C2=O
CC(CCCC(C)(C)O)C1CCC2C1(CCCC2=CC=C3CC(CC(C3=C)O)O)C
CC1=C(C(=C(C2=C1OC(CC2)(C)CCCC(C)CCCC(C)CCCC(C)C)C)OC(=O)C)C
CC(C(=O)O)N.C1=CC(=CC=C1CC(C(=O)O)N)O.C(CCN)CC(C(=O)O)N.C(CC(=O)O)C(C(=O)O)N
COC(=O)C=CC(=O)OC
CCC1C(=O)N(CC(=O)N(C(C(=O)NC(C(=O)N(C(C(=O)NC(C(=O)NC(C(=O)N(C(C(=O)N(C(C(=O)N(C(C(=O)N(C(C(=O)N1)C(C(C)CC=CC)O)C)C(C)C)C)CC(C)C)C)CC(C)C)C)C)C)CC(C)C)C)C(C)C)CC(C)C)C)C
Not found in PubChem (Status: 404)
CCCN(CCOC1=C(C=C(C=C1Cl)Cl)Cl)C(=O)N2C=CN=C2.C(S(=O)(=O)[O-])(I)I.[Na+]
CCCCCCCCC1=CC=C(C=C1)CCC(CO)(CO)N
CCC1=C(C=CC(=C1)C(=NOCC2=CC(=C(C=C2)C3CCCCC3)C(F)(F)F)C)CN4CC(C4)C(=O)O
C1=CN=C(C=N1)NNC(=O)C=CN2C=NC(=N2)C3=CC(=CC(=C3)C(F)(F)F)C(F)(F)F
C1=NC(=NN1C2C(C(C(O2)CO)O)O)C(=O)N
CN1C(=O)N2C=NC(=C2N=N1)C(=O)N
C1=NC2=C(C(=O)N1)N=CN2C3C(C(C(O3)CO)O)O
C1CC(=O)NC(=O)C1N2CC3=C(C2=O)C=CC=C3N
C1=C(NC=N1)CC(C(=O)O)N
CC1=C(C2=CC=CC=C2N1)CCNCC3=CC=C(C=C3)C=CC(=O)NO
CCCN=C1N(C(=O)C(=CC2=CC(=C(C=C2)OCC(CO)O)Cl)S1)C3=CC=CC=C3C
CC(C)OC1=C(C=C(C=C1)C2=NC(=NO2)C3=C4CCC(C4=CC=C3)NCCO)C#N
CC1CCC(CN1C(=O)C=C)NC2=NC=NC3=C2C=CN3
CCC=CCC=CCC=CCC=CCC=CCCCC(=O)O
C(CC(=O)N)C(C(=O)O)N
CCCCC(C(=O)NC(CCC(=O)O)C(=O)NC(CC1=CN=CN1)C(=O)NC(CC2=CC=CC=C2)C(=O)NC(CCCNC(=N)N)C(=O)NC(CC3=CNC4=CC=CC=C43)C(=O)NCC(=O)NC(CCCCN)C(=O)N5CCCC5C(=O)NC(C(C)C)C(=O)N)NC(=O)C(CO)NC(=O)C(CC6=CC=C(C=C6)O)NC(=O)C(CO)NC(=O)C
COCCOC(=O)NCCCCC(CCN1CCCC1C(=O)O)NC(=O)OCCOC
CCO
CC(C)C(C(=O)N1CCCC1C(=O)NC2=CC=C(C=C2)C3CCC(N3C4=CC=C(C=C4)C(C)(C)C)C5=CC=C(C=C5)NC(=O)C6CCCN6C(=O)C(C(C)C)NC(=O)OC)NC(=O)OC
C(C(C1C(=C(C(=O)O1)O)O)O)O
CC(C)C(C(=O)N1CCCC1C2=NC=C(N2)C3=CC4=C(C=C3)N5C(OC6=C(C5=C4)C=CC(=C6)C7=CN=C(N7)C8CCCN8C(=O)C(C(C)C)NC(=O)OC)C9=CC=CC=C9)NC(=O)OC
C1C2C1C3C=CC2C4C3C(=O)N(C4=O)NC(=O)C5=CC=C(C=C5)C(F)(F)F
CC(C)OC(=O)C(C)NP(=O)(OCC1C(C(C(O1)N2C=CC(=O)NC2=O)(C)F)O)OC3=CC=CC=C3
CC(C)C(C(=O)N1CC2(CC2)CC1C3=NC=C(N3)C4=CC5=C(C=C4)C6=C(C5(F)F)C=C(C=C6)C7=CC8=C(C=C7)N=C(N8)C9C1CCC(C1)N9C(=O)C(C(C)C)NC(=O)OC)NC(=O)OC
C1=C(C=C(C(=C1O)O)O)C(=O)OC2=CC(=CC(=C2O)O)C(=O)OCC3C(C(C(C(O3)OC(=O)C4=CC(=C(C(=C4)OC(=O)C5=CC(=C(C(=C5)O)O)O)O)O)OC(=O)C6=CC(=C(C(=C6)OC(=O)C7=CC(=C(C(=C7)O)O)O)O)O)OC(=O)C8=CC(=C(C(=C8)OC(=O)C9=CC(=C(C(=C9)O)O)O)O)O)OC(=O)C1=CC(=C(C(=C1)OC(=O)C1=CC(=C(C(=C1)O)O)O)O)O
CC(=O)OCC(CCN1C=NC2=CN=C(N=C21)N)COC(=O)C
CC1=C(C(=CC=C1)C)OCC(=O)NC(CC2=CC=CC=C2)C(CC(CC3=CC=CC=C3)NC(=O)C(C(C)C)N4CCCNC4=O)O
CCC(CC)OC1C=C(CC(C1NC(=O)C)N)C(=O)OCC
COC(=O)OCOC1=C2C(=O)N3CCOCC3N(N2C=CC1=O)C4C5=C(CSC6=CC=CC=C46)C(=C(C=C5)F)F
C1CC(OC1CO)N2C=NC3=C2N=CNC3=O
CC(C)CN1C=NC2=C1C3=CC=CC=C3N=C2N
[B]
CC1COC2N1C(=O)C3=C(C(=O)C(=CN3C2)C(=O)NCC4=C(C=C(C=C4)F)F)O
CC(C)(C)C(=O)OCOP(=O)(COCCN1C=NC2=C(N=CN=C21)N)OCOC(=O)C(C)(C)C
C1CC2CC1N3C(O2)CN4C=C(C(=O)C(=C4C3=O)O)C(=O)NCC5=C(C=C(C=C5F)F)F
CC1CCC2C(C(OC3C24C1CCC(O3)(OO4)C)OC(=O)CCC(=O)O)C
CC1=C2C(=NC=C1)N(C3=C(C=CC=N3)C(=O)N2)C4CC4
CC(C)C(C(=O)OCC(CO)OCN1C=NC2=C1N=C(NC2=O)N)N
CC1=C(C=CC=C1O)C(=O)NC(CSC2=CC=CC=C2)C(CN3CC4CCCCC4CC3C(=O)NC(C)(C)C)O
C(=O)(O)P(=O)(O)O
CC1CCOC2N1C(=O)C3=C(C(=O)C(=CN3C2)C(=O)NCC4=C(C=C(C=C4)F)F)O
C1CC1NC2=C3C(=NC(=N2)N)N(C=N3)C4CC(C=C4)CO
CC(C)C(CO)N1C=C(C(=O)C2=C1C=C(C(=C2)CC3=C(C(=CC=C3)Cl)F)OC)C(=O)O
CCCN(CCOC1=C(C=C(C=C1Cl)Cl)Cl)C(=O)N2C=CN=C2.C(S(=O)(=O)[O-])(I)I.[Na+]
C1C2CC3CC1CC(C2)(C3)N
C1=NC(=NN1C2C(C(C(O2)CO)O)O)C(=O)N
CC(C)OC(=O)C(C)NP(=O)(COC(C)CN1C=NC2=C(N=CN=C21)N)OC3=CC=CC=C3
CC(C)C1=NC(=CS1)CN(C)C(=O)NC(C(C)C)C(=O)NC(CC2=CC=CC=C2)CC(C(CC3=CC=CC=C3)NC(=O)OCC4=CN=CS4)O
CC(=O)NC1C(C=C(OC1C(C(CO)O)O)C(=O)O)N=C(N)N
CNC(=O)C1=NC=CC(=C1)OC2=CC=C(C=C2)NC(=O)NC3=CC(=C(C=C3)Cl)C(F)(F)F
C1C(C(OC1N2C=C(C(=O)NC2=O)C(F)(F)F)CO)O
C1=NC2=C(N1COCCO)N=C(NC2=O)N
C=C1C(CC(C1CO)O)N2C=NC3=C2N=C(NC3=O)N
C1=NC2=C(N1COC(CO)CO)N=C(NC2=O)N
COC1=CC(=CC(=C1OC)OC)C2C3C(COC3=O)C(C4=CC5=C(C=C24)OCO5)O
CC1=NN=C(O1)C(=O)NC(C)(C)C2=NC(=C(C(=O)N2C)O)C(=O)NCC3=CC=C(C=C3)F
CN1C(=NNC1=O)CN2C=CC(=C(C2=O)OC3=CC(=CC(=C3)C#N)Cl)C(F)(F)F
C1CCC(CC1)N2CCN(CC2)C3=C(N=C(S3)NC(=O)C4=CC(=C(N=C4)N5CCC(CC5)C(=O)O)Cl)C6=CC(=CS6)Cl
CC1=NN(C=N1)C2=NC=C(C3=C2N(C=C3C(=O)C(=O)N4CCN(CC4)C(=O)C5=CC=CC=C5)COP(=O)(O)O)OC
CC(C)CN(CC(C(CC1=CC=CC=C1)NC(=O)OC2CCOC2)OP(=O)(O)O)S(=O)(=O)C3=CC=C(C=C3)N
CC(C(C(=O)N1CCCC1C2=NC3=C(N2)C=C(C(=C3)F)C4CCC(N4C5=CC(=C(C(=C5)F)N6CCC(CC6)C7=CC=C(C=C7)F)F)C8=CC9=C(C=C8F)N=C(N9)C1CCCN1C(=O)C(C(C)OC)NC(=O)OC)NC(=O)OC)OC
CC1(C2C1C(N(C2)C(=O)C(C(C)(C)C)NC(=O)C(F)(F)F)C(=O)NC(CC3CCNC3=O)C#N)C
CC1=CC(=CC(=C1OC2=NC(=NC(=C2Br)N)NC3=CC=C(C=C3)C#N)C)C#N
CC1(CC1)S(=O)(=O)NC(=O)C2(CC2C(F)F)NC(=O)C3CC4CN3C(=O)C(NC(=O)OC5CCCC5OCC=CC(C6=NC7=CC=CC=C7N=C6O4)(F)F)C(C)(C)C
CCCC1(CC(=C(C(=O)O1)C(CC)C2=CC(=CC=C2)NS(=O)(=O)C3=NC=C(C=C3)C(F)(F)F)O)CCC4=CC=CC=C4
COC1=C(C=C(C=C1)C(F)(F)F)N2C(C3=C(C(=CC=C3)F)N=C2N4CCN(CC4)C5=CC(=CC=C5)OC)CC(=O)O
CC(C)NC1=NC2=CC(=C(C=C2N1C3C(C(C(O3)CO)O)O)Cl)Cl
CC(C12CC3CC(C1)CC(C3)C2)N
CCCCCCCCCCCCCCCCOCCCOP(=O)(COC(CN1C=CC(=NC1=O)N)CO)O
CC(C)(C)C1=CC(=CC(=C1OC)C2=CC3=C(C=C2)C=C(C=C3)NS(=O)(=O)C)N4C=CC(=O)NC4=O
CC(C)(C)C1C(=O)N2CC(CC2C(=O)NC3(CC3C=C)C(=O)NS(=O)(=O)C4CC4)OC5=NC6=C(C=CC(=C6)OC)N=C5CCCCCC7CC7OC(=O)N1
CCCCCCCCCCCCCCCCCCCCCCO
C1=NC2=C(N1CCC(CO)CO)N=C(NC2=O)N
CC1(C2CCC3(C(C2(CCC1OC4C(C(C(C(O4)C(=O)O)O)O)OC5C(C(C(C(O5)C(=O)O)O)O)O)C)C(=O)C=C6C3(CCC7(C6CC(CC7)(C)C(=O)O)C)C)C)C
CC(C)(C)C(C(=O)NC(CC1=CC=CC=C1)C(CN(CC2=CC=C(C=C2)C3=CC=CC=N3)NC(=O)C(C(C)(C)C)NC(=O)OC)O)NC(=O)OC
CC(C)(C)C(C(=O)N1CC(CC1C(=O)NC2(CC2C=C)C(=O)NS(=O)(=O)C3CC3)OC4=NC=C(C5=C4C=C(C=C5)Cl)OC)NC(=O)OC(C)(C)C
CCC1C2CN(C1C(=O)NC3(CC3C(F)F)C(=O)NS(=O)(=O)C4(CC4)C)C(=O)C(NC(=O)OC5CC5CCCCC(C6=NC7=C(C=C(C=C7)OC)N=C6O2)(F)F)C(C)(C)C
C1=CN(C(=O)N=C1N)CC(CO)OCP(=O)(O)O
CC(C)OC(=O)OCOP(=O)(COC(C)CN1C=NC2=C(N=CN=C21)N)OCOC(=O)OC(C)C
CC1CCC(N1C(=O)C(C(C)C)NC(=O)OC)C2=NC3=C(N2)C=CC4=CC5=C(C=C43)OCC6=C5C=CC(=C6)C7=CN=C(N7)C8CC(CN8C(=O)C(C9=CC=CC=C9)NC(=O)OC)COC
C1=CN(C(=O)N=C1N)C2C(C(C(O2)CO)O)O
CCC(CC)C(C1C(CC(C1O)C(=O)O)N=C(N)N)NC(=O)C
CC(C)C(C(=O)OCCOCN1C=NC2=C1N=C(NC2=O)N)N
C1CC1C#CC2(C3=C(C=CC(=C3)Cl)NC(=O)O2)C(F)(F)F
CC(C)(C#CC1=NC(=C(C=C1)C2=C3C(=C(C=C2)Cl)C(=NN3CC(F)(F)F)NS(=O)(=O)C)C(CC4=CC(=CC(=C4)F)F)NC(=O)CN5C6=C(C7CC7C6(F)F)C(=N5)C(F)(F)F)S(=O)(=O)C
CC(C)(C)NC(=O)C1CC2CCCCC2CN1CC(C(CC3=CC=CC=C3)NC(=O)C(CC(=O)N)NC(=O)C4=NC5=CC=CC=C5C=C4)O
CC(=O)OC1=CC=CC=C1C(=O)NC2=NC=C(S2)[N+](=O)[O-]
CC1=NN=C(N1C2CC3CCC(C2)N3CCC(C4=CC=CC=C4)NC(=O)C5CCC(CC5)(F)F)C(C)C
CC1=C2COC(=O)C2=C(C(=C1OC)CC=C(C)CCC(=O)O)O
C1=CN(C(=O)N=C1N)C2C(C(C(O2)CO)O)(F)F
CC1=CN(C(=O)NC1=O)C2C=CC(O2)CO
CCC(C)C(C(=O)NC(CCC(=O)O)C(=O)NC(CCC(=O)O)C(=O)NC(CO)C(=O)NC(CCC(=O)N)C(=O)NC(CC(=O)N)C(=O)NC(CCC(=O)N)C(=O)NC(CCC(=O)N)C(=O)NC(CCC(=O)O)C(=O)NC(CCCCN)C(=O)NC(CC(=O)N)C(=O)NC(CCC(=O)O)C(=O)NC(CCC(=O)N)C(=O)NC(CCC(=O)O)C(=O)NC(CC(C)C)C(=O)NC(CC(C)C)C(=O)NC(CCC(=O)O)C(=O)NC(CC(C)C)C(=O)NC(CC(=O)O)C(=O)NC(CCCCN)C(=O)NC(CC1=CNC2=CC=CC=C21)C(=O)NC(C)C(=O)NC(CO)C(=O)NC(CC(C)C)C(=O)NC(CC3=CNC4=CC=CC=C43)C(=O)NC(CC(=O)N)C(=O)NC(CC5=CNC6=CC=CC=C65)C(=O)NC(CC7=CC=CC=C7)C(=O)N)NC(=O)C(CC(C)C)NC(=O)C(CO)NC(=O)C(CC8=CN=CN8)NC(=O)C(C(C)CC)NC(=O)C(CC(C)C)NC(=O)C(CO)NC(=O)C(C(C)O)NC(=O)C(CC9=CC=C(C=C9)O)NC(=O)C
CCCCC1=NC(=C(N1CC2=CC=C(C=C2)C3=CC=CC=C3C4=NN=N[N-]4)CO)Cl.[K+]
CC1=CN=C(C=N1)C(=O)NC2CCCCCC=CC3CC3(NC(=O)C4CC(CN4C2=O)OC5=NC6=CC=CC=C6C7=CC=CC=C75)C(=O)NS(=O)(=O)C8CC8
CC1=C(C2=C(N1C(=O)C3=CC=C(C=C3)Cl)C=CC(=C2)OC)CC(=O)O
CC(C)(C)NC(=O)C1CN(CCN1CC(CC(CC2=CC=CC=C2)C(=O)NC3C(CC4=CC=CC=C34)O)O)CC5=CN=CC=C5
C1C(OC(S1)CO)N2C=CC(=NC2=O)N
CC1=CC(=CC(=C1NC2=NC(=NC=C2)NC3=CC=C(C=C3)C#N)C)C=CC#N
C1=CC(=CC=C1NC(=NC(=NCCCCCCN=C(N)N=C(N)NC2=CC=C(C=C2)Cl)N)N)Cl
CCO
CCCCCCCCCCCCCCCC[N+]1=CC=CC=C1
CC(C)O
CCCCCCC1=C(C=C(C=C1)O)O
CC1=CC(=CC(=C1Cl)C)O
CCCCCCCCCC[N+](C)(C)CCCCCCCCCC
CCCCCCCCCCCCC[N+](C)(C)CC1=CC=CC=C1
II
C=CN1CCCC1=O.II
CCCCCCCCCCCCCCCC[N+](C)(C)C
OO
CC(C)(C)CC(C)(C)C1=CC=C(C=C1)OCCOCC[N+](C)(C)CC2=CC=CC=C2
O=[Si]=O
CC1=CC(=C(C=C1)C(C)C)O
[O-]Cl
C1=CC(=C(C=C1Cl)O)OC2=C(C=C(C=C2)Cl)Cl
S1SSSSSSS1
CCN(CC)CC(=O)NC1=C(C=CC=C1C)C
C1N2CN3CN1CN(C2)C3
B(=O)OO
C(C(C(=O)O)O)(C(=O)O)O
C(CN(CC(=O)O)CC(=O)O)N(CC(=O)O)CC(=O)O
C1=CC2=C(C(=C1)O)N=CC=C2
C1=CC(=CC=C1NC(=O)NC2=CC(=C(C=C2)Cl)Cl)Cl
CC1=NC2=C(C=C1)C(=CC(=C2O)Cl)Cl
C1=CC=C(C=C1)OC(=O)C2=CC=CC=C2O
CCN(CC)C1=CC=C(C=C1)C(=C2C=CC(=[N+](CC)CC)C=C2)C3=CC=CC=C3
CCCCCCCCCCCC(=O)NCCC[N+](C)(C)CC(=O)[O-]
[B]
C=CN1CCCC1=O
C1=CC=C(C=C1)OCCO
CC(=O)O
OP(=O)(O)[O-].[Na+]
CC1=C(C(CCC1)(C)C)C=CC(=CC=CC(=CCO)C)C
CC(C)CCCC(C)C1CCC2C1(CCCC2=CC=C3CC(CCC3=C)O)C
O=[Zn]
CC1CCC(C(C1)O)C(C)C
CCCCCCCCCCCC[N+](C)(C)CCOC1=CC=CC=C1
C1=CC(=CC2=NC3=C(C=CC(=C3)N)C=C21)N
C1=C(C=C(C(=C1[N+](=O)[O-])O)[N+](=O)[O-])[N+](=O)[O-]
COC(=O)C1=CC=CC=C1O
C1=CC=C(C=C1)O
CCOC(=O)C1=CC=C(C=C1)N
C(C(CO)([N+](=O)[O-])Br)O
C1(C(=O)NC(=O)N1)NC(=O)N
CCCCOC1=NC2=CC=CC=C2C(=C1)C(=O)NCCN(CC)CC
CCCCOC1=CC=C(C=C1)OCCCN2CCOCC2
CN(C)C1=CC2=C(C=C1)N=C3C=CC(=[N+](C)C)C=C3S2.[Cl-]
C[N+](C)(C)CCO.C1=CC=C(C(=C1)C(=O)O)[O-]
CN1C2CCC1CC(C2)OC(=O)C(CO)C3=CC=CC=C3
C1=CC(=S)N(C=C1)O
CC[Hg]SC1=CC=CC=C1C(=O)[O-].[Na+]
COC1=C(C=CC(=C1)CC=C)O
CC1(C2CCC1(C(=O)C2)C)C
[F-].[Na+]
CC1(C2CCC(O1)(CC2)C)C
C(NC(=O)N(CO)C1C(=O)N(C(=O)N1CO)CO)O
CC1(C2CCC1(C(=O)C2)C)C
CC1=CC(=CC=C1)N(C)C(=S)OC2=CC3=CC=CC=C3C=C2
CCCO
CN(C)C1=CC=C(C=C1)C(=C2C=CC(=[N+](C)C)C=C2)C3=CC=C(C=C3)N(C)C
ClCl
C(=O)(N)N.OO
COC1=CC=CC=C1O
C1=CC(=CC(=C1)O)O
CC1CCC(C(C1)O)C(C)C
[K+].[I-]
C(C(CO)O)O
C1C(C(C(C(C1N)OC2C(C(C(C(O2)CN)O)O)N)OC3C(C(C(O3)CO)OC4C(C(C(C(O4)CN)O)O)N)O)O)N
CC(C)NC(=O)C1=CC=C(C=C1)CNNC
CN(CCCl)CCCl
O[Se](=O)O
CC(=O)NC1CCC2=CC(=C(C(=C2C3=CC=C(C(=O)C=C13)OC)OC)OC)OC
CC(C1=NC=C(S1)C(=O)NC2=NC=C(C(=C2)C(F)(F)F)Cl)NC(=O)C3=C(C(=NC=N3)N)Cl
C1CCC(C1)C(CC#N)N2C=C(C=N2)C3=C4C=CNC4NC=N3
B(C(CC(C)C)NC(=O)C(CC1=CC=CC=C1)NC(=O)C2=NC=CN=C2)(O)O
CC1=CC=C(C=C1)C2=CC(=NN2C3=CC=C(C=C3)S(=O)(=O)N)C(F)(F)F
[Zn+2].[Zn+2].[Zn+2].[Zn+2]
C(CNCCNCCN)N
C(C1C(C(C(C(O1)OC2C(OC(C2O)(CO)O)CO)O)O)O)O
C1=CC=[N+](C(=C1)SSC2=CC=CC=[N+]2[O-])[O-]
C1C(C(OC1N2C=C(C(=O)NC2=O)C(F)(F)F)CO)O
CC1=CC2=C(C3C4C5C6=C(C(=C7C(=C6C(N4C(C(C2)N3C)O)COC(=O)C8(CS5)C9=C(CCN8)C2=C(N9)C=CC(=C2)OC)OCO7)C)OC(=O)C)C(=C1OC)O
CC(=CCCC(=CCO)C)C
CCN1C2=C3C=C(NC3=NC=C2CN(C1=O)C4=C(C(=CC(=C4F)OC)OC)F)CN5CCOCC5
CCCCC(=O)OCC(=O)C1(CC(C2=C(C1)C(=C3C(=C2O)C(=O)C4=C(C3=O)C=CC=C4OC)O)OC5CC(C(C(O5)C)O)NC(=O)C(F)(F)F)O
CC1CN(CC(O1)C)C2=NC=C(C=C2)NC(=O)C3=CC=CC(=C3C)C4=CC=C(C=C4)OC(F)(F)F
COC1=CC(=CC(=C1)C#CC2=NN(C3=NC=NC(=C23)N)C4CCN(C4)C(=O)C=C)OC
CN(C)CC1=CN(N=C1C2=CC=CC=C2)C3=NC(=NC=C3)NC4=C(C=C(C(=C4)NC(=O)C=C)N5CCOCC5)OC
COC1=C(C=C(C=C1)C2=CC(=O)C3=C(C=C(C=C3O2)O)O)O
CC(C(F)(F)F)NC1=NC(=NC(=N1)C2=NC(=CC=C2)Cl)NC(C)C(F)(F)F
CNCC1=CC=C(C=C1)C2=C3CCNC(=O)C4=C3C(=CC(=C4)F)N2
CCP(CC)CC.CC(=O)OCC1C(C(C(C(O1)[S-])OC(=O)C)OC(=O)C)OC(=O)C.[Au+]
C1=CN(C(=O)N=C1N)C2C(C(C(O2)CO)O)(F)F
CCC1=C2CN3C(=CC4=C(C3=O)COC(=O)C4(CC)O)C2=NC5=C1C=C(C=C5)OC(=O)N6CCC(CC6)N7CCCCC7
CC1=C(C2=C(N1C(=O)C3=CC=C(C=C3)Cl)C=CC(=C2)OC)CC(=O)O
CC1=CC(=CC=C1)N(C)C(=S)OC2=CC3=CC=CC=C3C=C2
C=CCCCCCCCCC(=O)O
C1=CC(=C(C=C1Cl)Cl)COC(CN2C=CN=C2)C3=C(C=C(C=C3)Cl)Cl
C1=CC=C(C=C1)C(C2=CC=CC=C2)(C3=CC=CC=C3Cl)N4C=CN=C4
CC(C)(C)C#CC=CCN(C)CC1=CC=CC2=CC=CC=C21
CC(C)(C)C1=CC=C(C=C1)CN(C)CC2=CC=CC3=CC=CC=C32
CCO
C1=CC=C(C(=C1)C(=O)O)O
CC(C)N1CCN(CC1)C2=CC=C(C=C2)OCC3COC(O3)(CN4C=NC=N4)C5=C(C=C(C=C5)Cl)Cl
CCC(C(C)O)N1C(=O)N(C=N1)C2=CC=C(C=C2)N3CCN(CC3)C4=CC=C(C=C4)OCC5CC(OC5)(CN6C=NC=N6)C7=C(C=C(C=C7)F)F
B1(C2=C(CO1)C=C(C=C2)F)O
CC1CC(=O)C=C(C12C(=O)C3=C(O2)C(=C(C=C3OC)OC)Cl)OC
C1=CC(=C(C=C1F)F)C(CN2C=NC=N2)(CN3C=NC=N3)O
B(O)(O)O
B1(OB2OB(OB(O1)O2)[O-])[O-].[Na+].[Na+]
CC1CC=CC=CC=CC=CC(CC2C(C(CC(O2)(CC(CC3C(O3)C=CC(=O)O1)O)O)O)C(=O)O)OC4C(C(C(C(O4)C)O)N)O
CCCCCOC1=CC=C(C=C1)C2=CC=C(C=C2)C3=CC=C(C=C3)C(=O)NC4CC(C(NC(=O)C5C(C(CN5C(=O)C(NC(=O)C(NC(=O)C6CC(CN6C(=O)C(NC4=O)C(C)O)O)C(C(C7=CC=C(C=C7)O)O)O)C(C)O)C)O)OCC[N+](C)(C)C)O
C1=CC(=C(C=C1Cl)Cl)C(CN2C=CN=C2)OCC3=C(SC=C3)Cl
CC1=CC(=O)N(C(=C1)C2CCCCC2)O
C1=CC(=CC=C1CSC(CN2C=CN=C2)C3=C(C=C(C=C3)Cl)Cl)Cl
CCCCCCCCCCCCCCCC[N+](C)(C)CCN(CC1=CC=C(C=C1)OC)C2=NC=CC=N2
CCCCCCCCCCCCCCCCOP(=O)([O-])OCC[N+](C)(C)C
CC(=O)N1CCN(CC1)C2=CC=C(C=C2)OCC3COC(O3)(CN4C=CN=C4)C5=C(C=C(C=C5)Cl)Cl
[B]
CC12CCC(=O)C=C1CCC3C2C(CC4(C3CCC4(C(=O)CO)O)C)O
C1=CC=C(C=C1)C(=O)O
C1=CC(=C(C(=C1)Cl)SC(CCC2=CC=C(C=C2)Cl)CN3C=CN=C3)Cl
CCCCCOC1=CC=C(C=C1)C2=CC(=NO2)C3=CC=C(C=C3)C(=O)NC4CC(C(NC(=O)C5C(C(CN5C(=O)C(NC(=O)C(NC(=O)C6CC(CN6C(=O)C(NC4=O)C(C)O)O)C(C(C7=CC(=C(C=C7)O)OS(=O)(=O)O)O)O)C(CC(=O)N)O)C)O)O)O
CC(C(CN1C=NC=N1)(C2=C(C=C(C=C2)F)F)O)N3CCC(=C)CC3
CCCCCOC1=CC=C(C=C1)C2=CC=C(C=C2)C3=CC=C(C=C3)C(=O)NC4CC(C(NC(=O)C5C(C(CN5C(=O)C(NC(=O)C(NC(=O)C6CC(CN6C(=O)C(NC4=O)C(C)O)O)C(C(C7=CC=C(C=C7)O)O)O)C(C)O)C)O)O)O
COC1=C(C=CC(=C1)CC=C)O
CC1C=CC=CCCC=CC=CC=CC=CC(CC2C(C(CC(O2)(CC(C(CCC(CC(CC(CC(=O)OC(C(C1O)C)C)O)O)O)O)O)O)O)C(=O)O)OC3C(C(C(C(O3)C)O)N)O
CC1=CC(=C(C=C1)C(C)C)O
CC(C1=NC(=CS1)C2=CC=C(C=C2)C#N)C(CN3C=[N+](C=N3)C(C)OC(=O)N(C)C4=C(C=CC=N4)COC(=O)CNC)(C5=C(C=CC(=C5)F)F)O
C1=CC(=C(C=C1Cl)Cl)CON=C(CN2C=CN=C2)C3=C(C=C(C=C3)Cl)Cl
CCC(C)N1C(=O)N(C=N1)C2=CC=C(C=C2)N3CCN(CC3)C4=CC=C(C=C4)OCC5COC(O5)(CN6C=NC=N6)C7=C(C=C(C=C7)Cl)Cl
CCC(C)CC(C)CCCCCCCCC(=O)NC1CC(C(NC(=O)C2C(CCN2C(=O)C(NC(=O)C(NC(=O)C3CC(CN3C(=O)C(NC1=O)C(C)O)O)C(C(C4=CC=C(C=C4)O)O)O)C(CCN)O)O)NCCN)O
S=[Se]
CC1C=CC=CC=CC=CC=CC=CC=CC(CC2C(C(CC(O2)(CC(CC(C(CCC(CC(CC(=O)OC(C(C1O)C)C)O)O)O)O)O)O)O)C(=O)O)OC3C(C(C(C(O3)C)O)N)O
C1=CC(=CC=C1COC(CN2C=CN=C2)C3=C(C=C(C=C3)Cl)Cl)Cl
CC(C)C(C)C1(CCC2(C3CCC4C5(COCC4(C3=CCC2(C1C(=O)O)C)CC(C5OCC(C)(C(C)(C)C)N)N6C(=NC=N6)C7=CC=NC=C7)C)C)C
C1=CC(=S)N(C=C1)O
CN1C(=O)C=C(S1)Cl
CC[Hg]SC1=CC=CC=C1C(=O)[O-].[Na+]
CC1CCC2CC(C(=CC=CC=CC(CC(C(=O)C(C(C(=CC(C(=O)CC(OC(=O)C3CCCCN3C(=O)C(=O)C1(O2)O)C(C)CC4CCC(C(C4)OC)O)C)C)O)OC)C)C)C)OC
C1=CC=[N+](C(=C1)SSC2=CC=CC=[N+]2[O-])[O-]
C1=CC(=CC=C1C(=N)N)OCCCCCOC2=CC=C(C=C2)C(=N)N
CC(=O)N1CCN(CC1)C2=CC=C(C=C2)OCC3COC(O3)(CN4C=CN=C4)C5=C(C=C(C=C5)Cl)Cl
O=[Zn]
C1=CC2=C(C(=C(C=C2Cl)I)O)N=C1
C1=NC(=O)NC(=C1F)N
CN(CC=CC1=CC=CC=C1)CC2=CC=CC3=CC=CC=C32
[Ag]
CN(C)C1=CC=C(C=C1)C(=C2C=CC(=[N+](C)C)C=C2)C3=CC=C(C=C3)N(C)C
C1=CC2=C(C(=C1)Cl)SC=C2COC(CN3C=CN=C3)C4=C(C=C(C=C4)Cl)Cl
C1=CC(=CC=C1C2=CN=C(C=C2)C(C(CN3C=NN=N3)(C4=C(C=C(C=C4)F)F)O)(F)F)OCC(F)(F)F
CC(C1=NC=NC=C1F)C(CN2C=NC=N2)(C3=C(C=C(C=C3)F)F)O
C1C(SC(=C(C#N)N2C=CN=C2)S1)C3=C(C=C(C=C3)Cl)Cl
C1CS(=O)(=O)NCN1CN2CCS(=O)(=O)NC2
CC1C(C(C(O1)OC2C(C(C(C(C2O)O)N=C(N)N)O)N=C(N)N)OC3C(C(C(C(O3)CO)O)O)NC)(C=O)O
C1C(C(C(C(C1N)OC2C(C(C(C(O2)CN)O)O)N)OC3C(C(C(O3)CO)OC4C(C(C(C(O4)CN)O)O)N)O)O)N
CC1=C2COC(=O)C2=C(C(=C1OC)CC=C(C)CCC(=O)O)O
C=CN1CCCC1=O.II
CC1CCC(C(C1)O)C(C)C
CCCCCCCCCCCCC[N+](C)(C)CC1=CC=CC=C1
CC1CCC(C(C1)O)C(C)C
CC(C1=CC(=C(C=C1)C2=CC=CC=C2)F)C(=O)O
CC1(OC2CC3C4CC(C5=CC(=O)C=CC5(C4C(CC3(C2(O1)C(=O)CO)C)O)C)F)C
CC1CC2C3CC(C4=CC(=O)C=CC4(C3(C(CC2(C1C(=O)CO)C)O)Cl)C)F
C1=CC=C(C(=C1)CC(=O)OCC(=O)O)NC2=C(C=CC=C2Cl)Cl
C1=CC(=C(C=C1N=NC2=CC(=C(C=C2)O)C(=O)O)C(=O)O)O
CC1=C(C(=C(C=C1)Cl)NC2=CC=CC=C2C(=O)O)Cl
C1CN(CCC1(C2=CC=CC=C2)C(=O)O)CCC(C#N)(C3=CC=CC=C3)C4=CC=CC=C4
CC(C1=CC(=CC=C1)OC2=CC=CC=C2)C(=O)O
CC12CCC(=O)C=C1CCC3C2C(CC4(C3CCC4(C(=O)CS)O)C)O
CC(C1=CC2=C(C=C1)C=C(C=C2)OC)C(=O)O
CCC(=O)NS(=O)(=O)C1=CC=C(C=C1)C2=C(ON=C2C3=CC=CC=C3)C
CC(=O)CCC1=CC2=C(C=C1)C=C(C=C2)OC
C1=CC=NC(=C1)NS(=O)(=O)C2=CC=C(C=C2)N=NC3=CC(=C(C=C3)O)C(=O)O
C1C(C(C(C(C1N)OC2C(C(C(C(O2)CO)O)O)N)OC3C(C(C(O3)CO)OC4C(C(C(C(O4)CN)O)O)N)O)O)N
CCCCC(C)(CC=CC1C(CC(=O)C1CCCCCCC(=O)OC)O)O
C1=CC(=CC=C1C(=O)NCCC(=O)O)N=NC2=CC(=C(C=C2)O)C(=O)O
C1CN2C(=CC=C2C(=O)C3=CC=CC=C3)C1C(=O)O
CC1=C(C(=NO1)C2=CC=CC=C2)C3=CC=C(C=C3)S(=O)(=O)N
CCC1C=C(C(CC=CC=C(C(=O)OC(CC=C(C=C(C1OC2C(C(C(C(O2)(C)C)OC(=O)C(C)C)O)O)C)C)C(C)O)COC3C(C(C(C(O3)C)OC(=O)C4=C(C(=C(C(=C4O)Cl)O)Cl)CC)O)OC)O)C
CC(C)(C(C(=O)O)N)S
CC1CC=CC=CC=CC=CC(CC2C(C(CC(O2)(CC(CC3C(O3)C=CC(=O)O1)O)O)O)C(=O)O)OC4C(C(C(C(O4)C)O)N)O
CC1=C(C(=CC=C1)NC2=CC=CC=C2C(=O)O)C
CC12CC(=O)C3C(C1CCC2(C(=O)CO)O)CCC4=CC(=O)C=CC34C
CCCC1OC2CC3C4CCC5=CC(=O)C=CC5(C4C(CC3(C2(O1)C(=O)CO)C)O)C
CC1C=CC=C(C(=O)NC2=C(C3=C(C4=C(C(=C3O)C)OC(C4=O)(OC=CC(C(C(C(C(C(C1O)C)O)C)OC(=O)C)C)OC)C)C5=C2N6C=CC(=CC6=N5)C)O)C
CC12CCC(=O)C=C1CCC3C2C(CC4(C3CCC4(C(=O)CO)O)C)O
CN1C(=C(C2=CC=CC=C2S1(=O)=O)O)C(=O)NC3=CC=CC=N3
CC(=O)NC1C(C(C(OC1O)OS(=O)(=O)O)O)OC2C(C(C(C(O2)C(=O)O)O)O)O
CC1CC2C3CC(C4=CC(=O)C=CC4(C3(C(CC2(C1(C(=O)CO)O)C)O)F)C)F
CC(C)C(=O)OCC(=O)C12C(CC3C1(CC(C4C3CCC5=CC(=O)C=CC45C)O)C)OC(O2)C6CCCCC6
CC(C)CC1=CC=C(C=C1)C(C)C(=O)O
CC(C1=CC=C(C=C1)C(=O)C2=CC=CS2)C(=O)O
C1=CC2=C(C(=C1)OCC(COC3=CC=CC4=C3C(=O)C=C(O4)C(=O)O)O)C(=O)C=C(O2)C(=O)O
CCC1=C2C(=CC=C1)C3=C(N2)C(OCC3)(CC)CC(=O)O
CC1=CN=C(S1)NC(=O)C2=C(C3=CC=CC=C3S(=O)(=O)N2C)O
CC1C=CC=CCCC=CC=CC=CC=CC(CC2C(C(CC(O2)(CC(C(CCC(CC(CC(CC(=O)OC(C(C1O)C)C)O)O)O)O)O)O)O)C(=O)O)OC3C(C(C(C(O3)C)O)N)O
CC1=C(C=CN=C1CS(=O)C2=NC3=CC=CC=C3N2)OCCCOC
CCC(=O)OCC(=O)C1(C(CC2C1(CC(C3(C2CCC4=CC(=O)C=CC43C)Cl)O)C)C)OC(=O)CC
CN(C)CCOC(C1=CC=CC=C1)C2=CC=CC=C2
CC1=CC=C(C=C1)C2=CC(=NN2C3=CC=C(C=C3)S(=O)(=O)N)C(F)(F)F
CN(C)C(=O)C(CCN1CCC(CC1)(C2=CC=C(C=C2)Cl)O)(C3=CC=CC=C3)C4=CC=CC=C4
CC1C=CC=CC=CC=CC=CC=CC=CC(CC2C(C(CC(O2)(CC(CC(C(CCC(CC(CC(=O)OC(C(C1O)C)C)O)O)O)O)O)O)O)C(=O)O)OC3C(C(C(C(O3)C)O)N)O
CC1C=CC=C(C(=O)NC2=CC(=C3C(=C2O)C(=C(C4=C3C(=O)C(O4)(OC=CC(C(C(C(C(C(C1O)C)O)C)OC(=O)C)C)OC)C)C)O)O)C
CC(=O)OC1=CC=CC=C1C(=O)O
CCP(CC)CC.CC(=O)OCC1C(C(C(C(O1)[S-])OC(=O)C)OC(=O)C)OC(=O)C.[Au+]
CC1CC2C3CCC4=CC(=O)C=CC4(C3(C(CC2(C1(C(=O)CO)O)C)O)F)C
CC1=C(C2=C(C1=CC3=CC=C(C=C3)S(=O)C)C=CC(=C2)F)CC(=O)O
C(C1C(C(C(C(O1)O)N)O)O)O
O.O.[O-]S(=O)(=S)[O-].[O-]S(=O)(=S)[O-].[Na+].[Na+].[Na+].[Au+]
CC1CC2C3CCC4=CC(=O)C=CC4(C3(C(CC2(C1C(=O)CO)C)O)F)C
CC1=CC=C(C=C1)C(=O)C2=CC=C(N2C)CC(=O)O
C1=CC(=C(C=C1N)C(=O)O)O
C1=CC=C(C=C1)C2=C(OC(=N2)CCC(=O)O)C3=CC=CC=C3
CC(C1=CC(=CC=C1)C(=O)C2=CC=CC=C2)C(=O)O
[C]
COC1=CC=CC=C1CNC(=O)C2=NOC(=C2)C3=CC=CC=C3
CCOC(=O)C1(CCN(CC1)CCC(C#N)(C2=CC=CC=C2)C3=CC=CC=C3)C4=CC=CC=C4
CC1=C(C2=C(N1C(=O)C3=CC=C(C=C3)Cl)C=CC(=C2)OC)CC(=O)O
CC1=CN=C(C(=C1OC)C)CS(=O)C2=NC3=C(N2)C=C(C=C3)OC
CC1C(C(CC(O1)OC2C(C(C(OC2OC3=C4C=C5C=C3OC6=C(C=C(C=C6)C(C(C(=O)NC(C(=O)NC5C(=O)NC7C8=CC(=C(C=C8)O)C9=C(C=C(C=C9O)O)C(NC(=O)C(C(C1=CC(=C(O4)C=C1)Cl)O)NC7=O)C(=O)O)CC(=O)N)NC(=O)C(CC(C)C)NC)O)Cl)CO)O)O)(C)N)O
CC1=CC(=CC(=C1CC(C(=O)N(CC2=CC(=C(C=C2)OC)C(=O)O)C(C)C3=NC=C(N3)C4=CC=CC=C4)N)C)C(=O)N
C1=CC=C(C(=C1)CC(=O)O)NC2=C(C=CC=C2Cl)Cl
CC1C(C(C(O1)OC2C(C(C(C(C2O)O)N=C(N)N)O)N=C(N)N)OC3C(C(C(C(O3)CO)O)O)NC)(C=O)O
CCC(C)CCCC(=O)NC(CCN)C(=O)NC(C(C)O)C(=O)NC(CCN)C(=O)NC1CCNC(=O)C(NC(=O)C(NC(=O)C(NC(=O)C(NC(=O)C(NC(=O)C(NC1=O)CCN)CC(C)C)CC(C)C)CCN)CCN)C(C)O
CC12CC(C3C(C1CCC2(C(=O)CO)O)CCC4=CC(=O)C=CC34C)O
C1C(C(C(C(C1N)OC2C(C(C(C(O2)CN)O)O)N)OC3C(C(C(O3)CO)OC4C(C(C(C(O4)CN)O)O)N)O)O)N
CC1=C(C(=NN1)C)CCCOC2=C(C=CC(=C2)C(=O)O)F
CCOC1=CC=C(C=C1)CC2=C(C=CC(=C2)C3C(C(C(C(O3)SC)O)O)O)Cl
CS(=O)C
C(C(C(=O)[O-])O)(C(=O)O)O.[K+]
CCCN1CCCCC1C(=O)NC2=C(C=CC=C2C)C
CCCC(C(=O)OCC)NC(C)C(=O)N1C2CCCCC2CC1C(=O)O
C1CN=C(N1)NC2=C(C=C(C=C2Cl)N)Cl
CC(C)(CCCCCC(CCCCCC(C)(C)C(=O)O)O)C(=O)O
CC(=O)NC1CCC2=CC(=C(C(=C2C3=CC=C(C(=O)C=C13)OC)OC)OC)OC
CC(C)C1=NC(=NC(=C1C=CC(CC(CC(=O)O)O)O)C2=CC=C(C=C2)F)N(C)S(=O)(=O)C
COC(=O)NC1=C(N=C(N=C1N)C2=NN(C3=C2C=C(C=N3)F)CC4=CC=CC=C4F)N
CC1=C(SC(=N1)C2=CC(=C(C=C2)OCC(C)C)C#N)C(=O)O
CC1=C(C(C(=C(N1)C)C(=O)OC(C)C)C2=CC(=CC=C2)[N+](=O)[O-])C(=O)OCCOC
CN1CCC23C4C(=O)CCC2(C1CC5=C3C(=C(C=C5)OC)O4)O
C1CCC2(C3CC4=C(C2(C1)CCN3CC5CCC5)C=C(C=C4)O)O
CC1=C(C(=CC=C1)C)OCC(C)N
CC12CCC3C(C1CCC2(C)O)CCC4C3(CC(=CO)C(=O)C4)C
C1=CC2=C(C=C1S(=O)(=O)O)C(=C(N2)C3=NC4=C(C3=O)C=C(C=C4)S(=O)(=O)O)O
C1=CC(=C(C=C1C2=C(C=C(C=C2)F)F)C(=O)O)O
C1CCC(CC1)N2C(=O)C(C(=O)N(C2=O)C3CCCCC3)C(=O)NCC(=O)O
[O-][Si]1(O[Si](O[Si](O1)([O-])[O-])([O-])[O-])[O-].[Na+].[Na+].[Zr+4]
CC12CCC(=O)C=C1C3CC3C4C2CCC5(C4C6CC6C57CCC(=O)O7)C
CC1=CC=C(C=C1)N(CC2=NCCN2)C3=CC(=CC=C3)O
C1CC2=CN=CN2C1C3=C(C=C(C=C3)C#N)F
C1CCN(CC1)C2=NC(=N)N(C(=C2)N)O
Cl[Tl]
CCN(CC)CCCC(C)NC1=C2C=CC(=CC2=NC=C1)Cl
CCC(C)C1C(=O)NC(C(=O)NC(C(=O)NC(CSSCC(C(=O)NC(C(=O)N1)CC2=CC=C(C=C2)O)N)C(=O)N3CCCC3C(=O)NC(CC(C)C)C(=O)NCC(=O)N)CC(=O)N)CCC(=O)N
CC1=C(C(=CC=C1)Cl)NC(=O)C2=CN=C(S2)NC3=CC(=NC(=N3)C)N4CCN(CC4)CCO
C1CC(CNC1)C2=CC=C(C=C2)N3C=C4C=CC=C(C4=N3)C(=O)N
CC(=O)OC(CC(=O)[O-])C[N+](C)(C)C
CCCCCCCCC=CCCCCCCCC(=O)O.C(CO)N
CCCCCCC(C)(C)C1=CC(=C2C3CC(=O)CCC3C(OC2=C1)(C)C)O
CC1=C(N=CN1)CSCCNC(=NC)NC#N
CC1=C(C(C(=C(N1)C)C(=O)OC)C2=CC=CC=C2[N+](=O)[O-])C(=O)OC
CCCCC1=NC=C(N1CC2=CC=C(C=C2)C(=O)O)C=C(CC3=CC=CS3)C(=O)O
CC(C1=CC(=C(C=C1)C2=CC=CC=C2)F)C(=O)O
CCC(C)C(=O)OC1CC(C=C2C1C(C(C=C2)C)CCC3CC(CC(=O)O3)O)C
CC1=C(C=C(C(=O)N1)C#N)C2=CC=NC=C2
CCCCCCCCCCCCOCCOCCOCCOCCOCCOCCOCCOCCOCCO
CC(C(C1=CC(=CC=C1)O)O)N
O=O
CCOC(=O)C(CCC1=CC=CC=C1)NC(C)C(=O)N2C3CCCCC3CC2C(=O)O
COC1=CC(=C(C=C1)OC)C(CNC(=O)CN)O
[C-]#N.[C-]#N.[C-]#N.[C-]#N.[C-]#N.N#[O+].[Fe+2]
C(C(=O)[O-])C(CC(=O)[O-])(C(=O)[O-])O.C(C(=O)[O-])C(CC(=O)[O-])(C(=O)[O-])O.[Zn+2].[Zn+2].[Zn+2]
CCNC(=O)C1CCCN1C(=O)C(CCCN=C(N)N)NC(=O)C(CC(C)C)NC(=O)C(CC(C)C)NC(=O)C(CC2=CC=C(C=C2)O)NC(=O)C(CO)NC(=O)C(CC3=CNC4=CC=CC=C43)NC(=O)C(CC5=CN=CN5)NC(=O)C6CCC(=O)N6
CC1=NC2(C(O1)CC3C2(CC(C4C3CCC5=CC(=O)C=CC45C)O)C)C(=O)COC(=O)C
C1CC(N(C1)C(=O)C2CSSCC(C(=O)NC(C(=O)NC(C(=O)NC(C(=O)NC(C(=O)N2)CC(=O)N)CCC(=O)N)CC3=CC=CC=C3)CC4=CC=C(C=C4)O)NC(=O)CNC(=O)CNC(=O)CN)C(=O)NC(CCCCN)C(=O)NCC(=O)N
CN(CCC1=CC=C(C=C1)NS(=O)(=O)C)CCOC2=CC=C(C=C2)NS(=O)(=O)C
CC12CCC(=O)C=C1CC(C3C24C(O4)CC5(C3CCC56CCC(=O)O6)C)C(=O)OC
CN(CCCNC(=O)C1CCCO1)C2=NC3=CC(=C(C=C3C(=N2)N)OC)OC
CC(=O)NC1C(C(C(OC1O)COS(=O)(=O)O)OC2C(C(C(C(O2)C(=O)O)OC3C(C(C(C(O3)CO)OC4C(C(C(C(O4)C(=O)O)OC5C(C(C(C(O5)COS(=O)(=O)O)O)O)NS(=O)(=O)O)O)OS(=O)(=O)O)OS(=O)(=O)O)NS(=O)(=O)O)O)OS(=O)(=O)O)O
CC(C(C1=CC(=C(C=C1)O)O)O)N
CCN(CC)CCOC(=O)C1=C(C=C(C=C1)N)Cl
CC(C)(C(=O)O)OC1=CC=C(C=C1)C(=O)C2=CC=C(C=C2)Cl
C1=C2C(=CC(=C1Cl)S(=O)(=O)N)S(=O)(=O)N=CN2
CC(C)C1=C(C=C(C(=C1)C2=NNC(=O)N2C3=CC4=C(C=C3)N(C=C4)C)O)OP(=O)(O)O
C1=CC=C2C(=C1)C=NN=C2NN
C1CC(C2=C(C=CC=N2)C(C1C3=C(C(=CC=C3)F)F)N)OC(=O)N4CCC(CC4)N5C6=C(NC5=O)N=CC=C6
CCCCC1(CS(=O)(=O)C2=C(C=C(C=C2)N(C)C)C(C1O)C3=CC=C(C=C3)OCC4=CC=C(C=C4)C[N+]56CCN(CC5)CC6)CCCC
[Cl-].[K+]
CCCC1=NC(=C2N1N=C(NC2=O)C3=C(C=CC(=C3)S(=O)(=O)N4CCN(CC4)CC)OCC)C
CC(C)N(CCC(C1=CC=CC=C1)(C2=CC=CC=N2)C(=O)N)C(C)C
CC1C(=O)NC(C(=O)NC(C(=O)NC(C(=O)NC(C(=O)NC(CSSCC(C(=O)N1)NC(=O)C(CCCN=C(N)N)NC(=O)C)C(=O)N)CC2=CNC3=CC=CC=C32)CCCN=C(N)N)CC4=CC=CC=C4)CC5=CN=CN5
CC(C)NCC(COC1=CC=C(C=C1)CCOCC2CC2)O
CC1=CC(=CC=C1)NC2=C(C=NC=C2)S(=O)(=O)NC(=O)NC(C)C
CC1NC2=CC(=C(C=C2C(=O)N1C3=CC=CC=C3C)S(=O)(=O)N)Cl
CC(=O)CCCCN1C(=O)C2=C(N=CN2C)N(C1=O)C
C1CN(CCN1)C2=NC3=CC=CC=C3OC4=C2C=C(C=C4)Cl
CNCCCN1C2=CC=CC=C2CCC3=CC=CC=C31
C(C(C(C(C(C(=O)[O-])O)O)O)O)O.C(C(C(C(C(C(=O)[O-])O)O)O)O)O.[Zn+2]
COC1C(C(C(C(O1)COS(=O)(=O)O)OC2C(C(C(C(O2)C(=O)O)OC3C(C(C(C(O3)COS(=O)(=O)O)OC4C(C(C(C(O4)C(=O)O)OC5C(C(C(C(O5)COS(=O)(=O)O)O)O)NS(=O)(=O)O)O)OS(=O)(=O)O)NS(=O)(=O)O)O)OS(=O)(=O)O)O)NS(=O)(=O)O
[Zn]
CC1=C(C(C(=C(N1)C)C(=O)OCC(C)C)C2=CC=CC=C2[N+](=O)[O-])C(=O)OC
CC(C)NCC(C1=CC=C(C=C1)NS(=O)(=O)C)O
CC(CC1=CC(=C(C=C1)O)O)(C(=O)O)N
C1=CC(=C(C(=C1)Cl)CC(=O)N=C(N)N)Cl
CNC(=O)C1=CN(N=C1)C2=NC(=C3C(=N2)N(C=N3)C4C(C(C(O4)CO)O)O)N
CC(C)(C[N+]#[C-])OC.CC(C)(C[N+]#[C-])OC.CC(C)(C[N+]#[C-])OC.CC(C)(C[N+]#[C-])OC.CC(C)(C[N+]#[C-])OC.CC(C)(C[N+]#[C-])OC.[Tc]
CC(C)CCCC(C)C1CCC2C1(CCCC2=CC=C3CC(CCC3=C)O)C
CC1=C(C(=C(C2=C1OC(CC2)(C)CCCC(C)CCCC(C)CCCC(C)C)C)OC(=O)C)C
CC1CC2C3CC(C4=CC(=O)C=CC4(C3(C(CC2(C1(C(=O)CCl)O)C)O)F)C)F
CC(C)C(CCCN(C)CCC1=CC(=C(C=C1)OC)OC)(C#N)C2=CC(=C(C=C2)OC)OC
CC1=C(C(C(=C(N1)C)C(=O)OC(C)C)C2=CC=CC3=NON=C32)C(=O)OC
C1=CC(=C(C=C1C(CN)O)O)O
CC(C)NCC(COC1=CC=CC2=C1C=CN2)O
CCCC(=O)NC1=CC(=C(C=C1)OCC(CNC(C)C)O)C(=O)C
CCC1=C(CN(C1=O)C(=O)NCCC2=CC=C(C=C2)S(=O)(=O)NC(=O)NC3CCC(CC3)C)C
CC(C)NCC(C1=CC(=C(C=C1)O)O)O
CC(C)(CC1=CC=CC=C1)N
COCCCCC(=NOCCN)C1=CC=C(C=C1)C(F)(F)F
C1CC2=C(C=CC(=C2)F)OC1C(CNCC(C3CCC4=C(O3)C=CC(=C4)F)O)O
C(OC(C(F)(F)F)C(F)(F)F)F
C1=CC=C(C=C1)OC(=O)C2=CC=CC=C2O
CC(C(=O)N1CCCC1C(=O)O)NC(CCC2=CC=CC=C2)C(=O)O
CCCCCC(CCC1C(CC2C1CC3=C(C2)C(=CC=C3)OCC(=O)O)O)O
CC1=CC=CC=C1OC(CCNC)C2=CC=CC=C2
CC(CCC1=CC=CC=C1)NCC(C2=CC(=C(C=C2)O)C(=O)N)O
CC1C(C(CC(O1)OC2CC(CC3=C2C(=C4C(=C3O)C(=O)C5=C(C4=O)C(=CC=C5)OC)O)(C(=O)CO)O)N)O
C1=CN=CC=C1N
CC(C)NCC(COC1=CC=C(C=C1)CCC(=O)OC)O
CCC(=C)C(=O)C1=C(C(=C(C=C1)OCC(=O)O)Cl)Cl
CCNC(=O)CCCC=CCC1C(CC(C1C=CC(CCC2=CC=CC=C2)O)O)O
CCOC(=O)C1=C(NC(=C(C1C2=CC=CC=C2Cl)C(=O)OC)C)COCCN
C1=CC(=CN=C1)C(=O)O
CCCC1=NC2=C(N1CC3=CC=C(C=C3)C4=CC=CC=C4C(=O)O)C=C(C=C2C)C5=NC6=CC=CC=C6N5C
C1NC2=CC(=C(C=C2S(=O)(=O)N1)S(=O)(=O)N)Cl
CCN(CC)CCNC(=O)C1=CC=C(C=C1)N
CN(C)CCC=C1C2=CC=CC=C2CCC3=CC=CC=C31
CN1C=NC2=C1C(=O)N(C(=O)N2C)C
CCN1C(=O)C(NC1=O)C2=CC=CC=C2
CCC(C)C.CCC(CC(C)C1=CC=CC=C1)C2=CC=C(C=C2)C[N+](C)(C)C.N.[Cl-]
CNCC(C1=CC(=CC=C1)O)O
COC1=C(C=C(C=C1)CC2=NC=CC3=CC(=C(C=C32)OC)OC)OC
CCOC(=O)C1=C(NC(=C(C1C2=C(C(=CC=C2)Cl)Cl)C(=O)OC)C)C
[K]
CCNC(=O)C1CCCN1C(=O)C(CCCN=C(N)N)NC(=O)C(CC(C)C)NC(=O)C(CC2=CN(C=N2)CC3=CC=CC=C3)NC(=O)C(CC4=CC=C(C=C4)O)NC(=O)C(CO)NC(=O)C(CC5=CNC6=CC=CC=C65)NC(=O)C(CC7=CN=CN7)NC(=O)C8CCC(=O)N8
C1=CC(=C(C(=C1)F)CN2C=C(N=N2)C(=O)N)F
C1CC(C2=CC=CC=C2C1)C3=NCCN3
CC(C)(C(=O)NC(CC1=CNC2=CC=CC=C21)C(=O)NC(CC3=CNC4=CC=CC=C43)NC=O)N
C1=CC(=C(C=C1C(C(C(=O)O)N)O)O)O
CCCCNC1=CC=C(C=C1)C(=O)OCCN(C)C
CCC1=CC2CC(C3=C(CN(C2)C1)C4=CC=CC=C4N3)(C5=C(C=C6C(=C5)C78CCN9C7C(C=CC9)(C(C(C8N6C)(C(=O)OC)O)OC(=O)C)CC)OC)C(=O)OC
C1CN(CCC1NC(=O)C2=CC=CC=C2C3=CC=C(C=C3)C(F)(F)F)CCCCC4(C5=CC=CC=C5C6=CC=CC=C64)C(=O)NCC(F)(F)F
CCC1C(C(C(N(CC(CC(C(C(C(C(C(=O)O1)C)OC2CC(C(C(O2)C)O)(C)OC)C)OC3C(C(CC(O3)C)N(C)C)O)(C)O)C)C)C)O)(C)O
CC1CC2C3CCC(C3(CC(C2(C4(C1=CC(=O)C=C4)C)F)O)C)(C(=O)C)O
CN1C2CCC1C(C(C2)OC(=O)C3=CC=CC=C3)C(=O)OC
C1CCN(CC1)CCOC2=CC=C(C=C2)C(=O)C3=C(SC4=C3C=CC(=C4)O)C5=CC=C(C=C5)O
[O-]S(=O)(=O)[O-].[Ba+2]
CCCC(CCC)C(=O)O
COC1=CC2=C(C=CN=C2C=C1)C(C3CC4CCN3CC4C=C)O
CC1(C(=O)N2C(C(=O)N3CCCC3C2(O1)O)CC4=CC=CC=C4)NC(=O)C5CN(C6CC7=CNC8=CC=CC(=C78)C6=C5)C
CN(C)CCCC1(C2=C(CO1)C=C(C=C2)C#N)C3=CC=C(C=C3)F
CC12CC(C3(C(C1CC(C2(C(=O)CO)O)O)CCC4=CC(=O)C=CC43C)F)O
CC12CCC(=O)C=C1CCC3C2C(CC4(C3CCC4(C(=O)CO)O)C)O
CC1=C(C=C(C=C1)NC(=O)C2=CC=C(C=C2)CN3CCN(CC3)C)NC4=NC=CC(=N4)C5=CN=CC=C5
CC1=C(C=C(C=C1)C2C(C(C(C(O2)CO)O)O)O)CC3=CC=C(S3)C4=CC=C(C=C4)F
CCOC(=O)C1=CN=CN1C(C)C2=CC=CC=C2
CC1=CC=C(C=C1)C2=CC(=NN2C3=CC=C(C=C3)S(=O)(=O)N)C(F)(F)F
CCCCOC1=CC=C(C=C1)C(=O)CCN2CCCCC2
CN1CCCC1CC2=CNC3=C2C=C(C=C3)CCS(=O)(=O)C4=CC=CC=C4
CCCCCN=C(N)NN=CC1=CNC2=C1C=C(C=C2)OC
CCC=CCC=CCC=CCC=CCC=CCCCC(=O)OCC
CC(=O)OC1CC2CCC3C(C2(CC1N4CCCCC4)C)CCC5(C3CC(C5OC(=O)C)[N+]6(CCCCC6)C)C
CCOC(=O)NC1CCC2C(C1)CC3C(C2C=CC4=NC=C(C=C4)C5=CC(=CC=C5)F)C(OC3=O)C
CCC1=NN(C(=O)N1CCOC2=CC=CC=C2)CCCN3CCN(CC3)C4=CC(=CC=C4)Cl
CCOC(=O)C(CCC1=CC=CC=C1)NC(C)C(=O)N2C3CCCC3CC2C(=O)O
CCCCCCCCC1=CC=C(C=C1)CCC(CO)(CO)N
CC(C)C1=C(C(=C(N1CCC(CC(CC(=O)O)O)O)C2=CC=C(C=C2)F)C3=CC=CC=C3)C(=O)NC4=CC=CC=C4
CCC(C)C(C(=O)NC(C)C(=O)NC(CC1=CNC2=CC=CC=C21)C(=O)NC(CC(C)C)C(=O)NC(C(C)C)C(=O)NC(CCCNC(=N)N)C(=O)NCC(=O)NC(CCCNC(=N)N)C(=O)NCC(=O)O)NC(=O)C(CC3=CC=CC=C3)NC(=O)C(CCC(=O)O)NC(=O)C(CCCCNC(=O)COCCOCCNC(=O)COCCOCCNC(=O)CCC(C(=O)O)NC(=O)CCCCCCCCCCCCCCCCC(=O)O)NC(=O)C(C)NC(=O)C(C)NC(=O)C(CCC(=O)N)NC(=O)CNC(=O)C(CCC(=O)O)NC(=O)C(CC(C)C)NC(=O)C(CC4=CC=C(C=C4)O)NC(=O)C(CO)NC(=O)C(CO)NC(=O)C(C(C)C)NC(=O)C(CC(=O)O)NC(=O)C(CO)NC(=O)C(C(C)O)NC(=O)C(CC5=CC=CC=C5)NC(=O)C(C(C)O)NC(=O)CNC(=O)C(CCC(=O)O)NC(=O)C(C)(C)NC(=O)C(CC6=CN=CN6)N
C1COCC1OC2=CC=C(C=C2)CC3=C(C=CC(=C3)C4C(C(C(C(O4)CO)O)O)O)Cl
CC(C)NCC(COC1=CC=C(C=C1)COCCOC(C)C)O
N(=O)O
CCN(CC)CC(=O)NC1=C(C=CC=C1C)C
CCC(C)C(=O)OC1CC(C=C2C1C(C(C=C2)C)CCC(CC(CC(=O)O)O)O)O
COC1=CC=CC=C1OCCNCC(COC2=CC=CC3=C2C4=CC=CC=C4N3)O
COCCCN1CCC(CC1)NC(=O)C2=CC(=C(C3=C2OCC3)N)Cl
CCC1=C2C(=CC=C1)C3=C(N2)C(OCC3)(CC)CC(=O)O
CCCCNC1=CC=C(C=C1)C(=O)OCCOCCOCCOCCOCCOCCOCCOCCOCCOC
CCOC(=O)C1=CC=C(C=C1)N
CCOC1=NC=C(C2=C1C(C(=C(N2)C)C(=O)N)C3=C(C=C(C=C3)C#N)OC)C
CC(C)NCC(COC1=CC=C(C=C1)CCOC)O
CN(C)CCC1=CNC2=C1C=C(C=C2)CC3COC(=O)N3
CC1C(OCCN1C)C2=CC=CC=C2
C1=NC(=C2C(=N1)N(C=N2)C3C(C(C(O3)CO)O)O)N
CC(C)N1C2=CC=CC=C2C(=C1C=CC(CC(CC(=O)O)O)O)C3=CC=C(C=C3)F
CC1=C(C(=CC=C1)C)NC(=O)C2CCCCN2C
CC1=CN=C(S1)NC(=O)C2=C(C3=CC=CC=C3S(=O)(=O)N2C)O
C1C(C2C(O1)C(CO2)O[N+](=O)[O-])O
CC[N+](C)(C)CC1=CC=CC=C1Br
CCC(C)C(C(=O)NC(CC1=CN=CN1)C(=O)N2CCCC2C(=O)NC(CC3=CC=CC=C3)C(=O)O)NC(=O)C(CC4=CC=C(C=C4)O)NC(=O)C(C(C)C)NC(=O)C(CCCN=C(N)N)NC(=O)C(CC(=O)O)N
CC12CCC3C(C1CCC2O)CCC4=C3C=CC(=C4)O
CC1C(C(CC(O1)OC2C(OC(CC2O)OC3C(OC(CC3O)OC4CCC5(C(C4)CCC6C5CC(C7(C6(CCC7C8=CC(=O)OC8)O)C)O)C)C)C)O)O
CC(C)(C)NCC(C1=CC(=C(C=C1)O)CO)O
CCCCCCCCCCCCCCOS(=O)(=O)[O-].[Na+]
CCCCCON=O
CC1=C(C2=C(CCC(O2)(C)CCCC(C)CCCC(C)CCCC(C)C)C(=C1O)C)C
CCCC(=O)OC1(CCC2C1(CC(C3C2CCC4=CC(=O)CCC34C)O)C)C(=O)CO
CCCC(=O)OC1(CCC2C1(CC(C3C2CCC4=CC(=O)CCC34C)O)C)C(=O)COC(=O)CC
CCCCC(=O)OC1(CCC2C1(CC(C3C2CCC4=CC(=O)CCC34C)O)C)C(=O)CO
CC1=CN(C(=O)NC1=O)C2CC(C(O2)COP(=S)(O)OC3CC(OC3COP(=S)(O)OC4CC(OC4COP(=S)(O)OC5C(OC(C5OCCOC)N6C=C(C(=O)NC6=O)C)COP(=S)(O)OC7C(OC(C7OCCOC)N8C=C(C(=O)NC8=O)C)COP(=S)(O)OC9C(OC(C9OCCOC)N1C=C(C(=NC1=O)N)C)COP(=S)(O)OC1C(OC(C1OCCOC)N1C=NC2=C1N=C(NC2=O)N)COP(=S)(O)OC1C(OC(C1OCCOC)N1C=NC2=C(N=CN=C21)N)CO)N1C=C(C(=NC1=O)N)C)N1C=C(C(=O)NC1=O)C)OP(=S)(O)OCC1C(CC(O1)N1C=NC2=C1N=C(NC2=O)N)OP(=S)(O)OCC1C(CC(O1)N1C=C(C(=O)NC1=O)C)OP(=S)(O)OCC1C(CC(O1)N1C=C(C(=NC1=O)N)C)OP(=S)(O)OCC1C(CC(O1)N1C=C(C(=NC1=O)N)C)OP(=S)(O)OCC1C(CC(O1)N1C=NC2=C(N=CN=C21)N)OP(=S)(O)OCC1C(CC(O1)N1C=NC2=C1N=C(NC2=O)N)OP(=S)(O)OCC1C(CC(O1)N1C=C(C(=NC1=O)N)C)OP(=S)(O)OCC1C(C(C(O1)N1C=C(C(=O)NC1=O)C)OCCOC)OP(=S)(O)OCC1C(C(C(O1)N1C=C(C(=O)NC1=O)C)OCCOC)OP(=S)(O)OCC1C(C(C(O1)N1C=C(C(=O)NC1=O)C)OCCOC)OP(=S)(O)OCC1C(C(C(O1)N1C=NC2=C(N=CN=C21)N)OCCOC)OP(=S)(O)OCC1C(C(C(O1)N1C=C(C(=O)NC1=O)C)OCCOC)O
O=[Mg]
C[N+]1(CCC(C1)OC(=O)C(C2CCCC2)(C3=CC=CC=C3)O)C
COC1=C(C=C(C=C1)Cl)C(=O)NCCC2=CC=C(C=C2)S(=O)(=O)NC(=O)NC3CCCCC3
CCOC(=O)C(CCC1=CC=CC=C1)NC(C)C(=O)N2CC3=CC=CC=C3CC2C(=O)O
C1CC1C2=NC3=CC=CC=C3C(=C2C=CC(CC(CC(=O)O)O)O)C4=CC=C(C=C4)F
N
[O-]S(=O)(=O)[O-].[Fe+2]
C(C(CO[N+](=O)[O-])O[N+](=O)[O-])O[N+](=O)[O-]
C1=CC(=C(C(=C1)Cl)Cl)C2=C(N=C(N=N2)N)N
CNC1CCC(C2=CC=CC=C12)C3=CC(=C(C=C3)Cl)Cl
CC1(C2CCC(C2)C1(C)NC)C
CCOC(=O)C(CCC1=CC=CC=C1)NC(C)C(=O)N2CC3=CC(=C(C=C3CC2C(=O)O)OC)OC
C1=CC=C(C=C1)O
CN(CCCN1CCC2=CC(=C(C=C2CC1=O)OC)OC)CC3CC4=CC(=C(C=C34)OC)OC
CC(=O)C(=O)O
CC(=O)SC1CC2=CC(=O)CCC2(C3C1C4CCC5(C4(CC3)C)CCC(=O)O5)C
CC(=O)NC1C(C(C(OC1O)COS(=O)(=O)O)OC2C(C(C(C(O2)C(=O)O)OC3C(C(C(C(O3)CO)OC4C(C(C(C(O4)C(=O)O)O)O)OS(=O)(=O)O)OS(=O)(=O)O)NS(=O)(=O)O)O)OS(=O)(=O)O)O
CC(=O)NC1C(C(C(OC1O)COS(=O)(=O)O)OC2C(C(C(C(O2)C(=O)O)OC3C(C(C(C(O3)CO)OC4C(C(C(C(O4)C(=O)O)O)O)OS(=O)(=O)O)OS(=O)(=O)O)NS(=O)(=O)O)O)OS(=O)(=O)O)O
C(C(F)(F)F)(C(F)(F)F)(F)F
CCC1C(C(C(C(=O)C(CC(C(C(C(C(C(=O)O1)C)OC2CC(C(C(O2)C)O)(C)OC)C)OC3C(C(CC(O3)C)N(C)C)O)(C)OC)C)C)O)(C)O
CC(=O)OCC[N+](C)(C)C
CC1=CC=CC=C1C(=O)NC2=CC(=C(C=C2)C(=O)N3CCCC(C4=C3C=CC(=C4)Cl)O)C
CCCCCC1=CC(=C2C3C=C(CCC3C(OC2=C1)(C)C)C)O
C1=CC(=CC=C1C(CC(=O)O)CN)Cl
CC(=O)N1CCN(CC1)C2=CC=C(C=C2)OCC3COC(O3)(CN4C=CN=C4)C5=C(C=C(C=C5)Cl)Cl
CCCCCC(C=CC1C(CC(=O)C1CCCCCCC(=O)O)O)O
C(CS)N
C1=CC(=C(C=C1CCN)O)O
CCCC1=NC(=C(N1CC2=CC=C(C=C2)C3=CC=CC=C3C4=NNN=N4)C(=O)O)C(C)(C)O
C1CC(N(C1)C(=O)C(CCCCN)NC(CCC2=CC=CC=C2)C(=O)O)C(=O)O
CN(C)CCOC(C1=CC=CC=C1)C2=CC=CC=C2
C(C(C1C(=C(C(=O)O1)[O-])O)O)O.C(C(C1C(=C(C(=O)O1)[O-])O)O)O.[Zn+2]
CC(CC1=CC=CC=C1)N
CC(C(C1=CC=CC=C1)O)N(C)C
CN1CC(=O)N2C(C1=O)CC3=C(C2C4=CC5=C(C=C4)OCO5)NC6=CC=CC=C36
CC(CCC1=CC=C(C=C1)O)NCCC2=CC(=C(C=C2)O)O
CC(C)(C)NCC(COC1=CC=CC2=C1CC(C(C2)O)O)O
C1CC(CCC1CN)C(=O)O
CCCCS(=O)(=O)NC(CC1=CC=C(C=C1)OCCCCC2CCNCC2)C(=O)O
CCN1CCCC1CNC(=O)C2=CC(=C(C=C2OC)N)S(=O)(=O)CC
C1=CC=C(C=C1)C2(C(=O)NC(=O)N2)C3=CC=CC=C3
CCC(C)C1C(=O)NCC(=O)NC(C(=O)NC(C(=O)NC(C(=O)NCC(=O)NC(C(=O)NCC(=O)NC(CSSCC(C(=O)NC(C(=O)NCC(=O)NC(C(=O)NC(C(=O)NC(C(=O)NC(C(=O)NC(C(=O)N1)CCCNC(=N)N)CC(=O)O)CC(C)C)CCCCN)CC(C)C)CC2=CC=CC=C2)NC(=O)CNC(=O)C(CCCCN)NC(=O)C(CO)NC(=O)C(CC(C)C)NC(=O)CNC(=O)C(CCCCN)NC(=O)C(CCCCN)NC(=O)C(CC(=O)N)NC(=O)C(C)NC(=O)CNC(=O)C(CCCCN)NC(=O)C(CC3=CC=C(C=C3)O)NC(=O)C(CCCCN)NC(=O)C(CCCNC(=N)N)NC(=O)C(C)NC(=O)C(CC(=O)N)NC(=O)C4CCCN4C(=O)C(CC5=CN=CN5)NC(=O)C(CCC(=O)O)NC(=O)C(CCC(=O)N)NC(=O)CNC(=O)C6CCCN6)C(=O)O)CC(C)C)CO)CCSC)CO
CCCCNC1=C(C(=CC(=C1)C(=O)O)S(=O)(=O)N)OC2=CC=CC=C2
CCCNCC(COC1=CC=CC=C1C(=O)CCC2=CC=CC=C2)O
[Xe]
CC1=CN=C(C=N1)C(=O)NCCC2=CC=C(C=C2)S(=O)(=O)NC(=O)NC3CCCCC3
CCCCCC(C=CC1C(CC2C1CC(=CCCCC(=O)O)O2)O)O
O=[Zn]
CCCCOC1=CC=C(C=C1)OCCCN2CCOCC2
C1CCN(CC1)C2=NC(=NC3=C2N=C(N=C3N4CCCCC4)N(CCO)CCO)N(CCO)CCO
CCCCC1=NC2(CCCC2)C(=O)N1CC3=CC=C(C=C3)C4=CC=CC=C4C5=NNN=N5
C1CCNC(C1)CNC(=O)C2=C(C=CC(=C2)OCC(F)(F)F)OCC(F)(F)F
CC1C(C(C(C(O1)OCC2C(C(C(C(O2)OC3=C(OC4=CC(=CC(=C4C3=O)O)O)C5=CC(=C(C=C5)O)O)O)O)O)O)O)O
C1CN=C(N1)CC2=CC=CC3=CC=CC=C32
C1=CC(=CC(=C1)Cl)C2=CC(=C(N=C2)C(=O)NCC(=O)O)O
CC(C)NCC(COC1=CC=CC2=CC=CC=C21)O
CCCC(=O)OCOC(=O)C1=C(NC(=C(C1C2=C(C(=CC=C2)Cl)Cl)C(=O)OC)C)C
CC(C)N(CCCCOCC(=O)NS(=O)(=O)C)C1=CN=C(C(=N1)C2=CC=CC=C2)C3=CC=CC=C3
CC1=NC2=C(N1)CCN(C3=CC=CC=C32)C(=O)C4=CC=C(C=C4)NC(=O)C5=CC=CC=C5C6=CC=CC=C6
C1COCC(=O)N1C2=CC=C(C=C2)N3CC(OC3=O)CNC(=O)C4=CC=C(S4)Cl
[N+](=O)([O-])[O-].[K+]
CC(=O)OC1CC2=CC=CC=C2N(C3=CC=CC=C13)C(=O)N
CCCCC1=C(C2=C(O1)C=CC(=C2)NS(=O)(=O)C)C(=O)C3=CC=C(C=C3)OCCCN(CCCC)CCCC
CC(=O)NC(COC)C(=O)NCC1=CC=CC=C1
CN1C2CCC1CC(C2)OC(=O)C(CO)C3=CC=CC=C3
CCCCOC1=NC2=CC=CC=C2C(=C1)C(=O)NCCN(CC)CC
C1CN=C(N1)NC2=C(C=CC=C2Cl)Cl
CNCC(C1=CC(=C(C=C1)O)O)O
CCCCC1=NC(=C(N1CC2=CC=C(C=C2)C3=CC=CC=C3C4=NNN=N4)CO)Cl
[Cr]
CNS(=O)(=O)CCC1=CC2=C(C=C1)NC=C2C3CCN(CC3)C
CNCC(C1=CC(=C(C=C1)O)O)O
CCN(CC)CCOC(=O)C1=CC=C(C=C1)N
[Li+].[Li+].C(=O)([O-])[O-]
CC(C(C1=CC=CC=C1)O)NC
CC(=O)OC1=CC=CC=C1C(=O)O
CC(C)N1C(=O)C=C(NC1=O)NC(C)C2=CC=CC=C2
[N]=O
[O-]S(=O)(=O)[O-].[Mg+2]
CNS(=O)(=O)CC1=CC2=C(C=C1)NC=C2CCN(C)C
CCCC1=NN(C2=C1N=C(NC2=O)C3=C(C=CC(=C3)S(=O)(=O)N4CCN(CC4)C)OCC)C
CC(C1=NCCN1)OC2=C(C=CC=C2Cl)Cl
CCOC(=O)C1=C(NC(=C(C1C2=CC=CC=C2Cl)C(=O)OC)C)COCCN
CC1=NC=C2N1C3=C(C=C(C=C3)Cl)C(=NC2)C4=CC=CC=C4F
CN1C2=C(C(=O)N(C1=O)C)NC=N2
CC1(C(=O)N2C(C(=O)N3CCCC3C2(O1)O)CC4=CC=CC=C4)NC(=O)C5CC6C(CC7=CNC8=CC=CC6=C78)N(C5)C
CN(CC(CO)O)C(=O)C1=C(C(=C(C(=C1I)C(=O)NCC(CO)O)I)NC(=O)COC)I
CC(C)OC(=O)C(C)(C)OC1=CC=C(C=C1)C(=O)C2=CC=C(C=C2)Cl
CCC(C)C(C(=O)NC(C)C(=O)NC(CC1=CNC2=CC=CC=C21)C(=O)NC(CC(C)C)C(=O)NC(C(C)C)C(=O)NC(CCCCN)C(=O)NCC(=O)NCC(=O)NCC(=O)O)NC(=O)C(CC3=CC=CC=C3)NC(=O)C(CCC(=O)O)NC(=O)C(CCCCN)NC(=O)C(C)NC(=O)C(C)NC(=O)C(CCC(=O)N)NC(=O)C(CCC(=O)O)NC(=O)C(CCC(=O)O)NC(=O)C(CC(C)C)NC(=O)C(CC4=CC=C(C=C4)O)NC(=O)C(CO)NC(=O)C(CO)NC(=O)C(C(C)C)NC(=O)C(CC(=O)O)NC(=O)C(CO)NC(=O)C(C(C)O)NC(=O)C(CC5=CC=CC=C5)NC(=O)C(C(C)O)NC(=O)CNC(=O)C(CCC(=O)O)NC(=O)CNC(=O)C(CC6=CNC=N6)N
CC(C)OC1=C(C=C(C=C1)C2=NC(=NO2)C3=C4CCC(C4=CC=C3)NCCO)C#N
CC1=NS(=O)(=O)C2=C(N1)C=CC(=C2)Cl
C1C=CN(C=C1C(=O)N)C2C(C(C(O2)COP(=O)(O)OP(=O)(O)OCC3C(C(C(O3)N4C=NC5=C(N=CN=C54)N)O)O)O)O
CC(=O)NC1C(C(C(OC1OCCCCC(=O)NCCCNC(=O)CCOCC(COCCC(=O)NCCCNC(=O)CCCCOC2C(C(C(C(O2)CO)O)O)NC(=O)C)(COCCC(=O)NCCCNC(=O)CCCCOC3C(C(C(C(O3)CO)O)O)NC(=O)C)NC(=O)CCCCCCCCCCC(=O)N4CC(CC4COP(=O)(O)O)O)CO)O)O
COC(=O)C(C1=CC=CC=C1Cl)N2CCC3=C(C2)C=CS3
C[N+]1(CCC2=CC(=C(C=C2C1CC3=CC(=C(C=C3)OC)OC)OC)OC)CCC(=O)OCCCCCOC(=O)CC[N+]4(CCC5=CC(=C(C=C5C4CC6=CC(=C(C=C6)OC)OC)OC)OC)C.C1=CC=C(C=C1)S(=O)(=O)[O-].C1=CC=C(C=C1)S(=O)(=O)[O-]
CC1CCC(CN1C(=O)C=C)NC2=NC=NC3=C2C=CN3
CC(C)C(CC1=CC(=C(C=C1)OC)OCCCOC)CC(C(CC(C(C)C)C(=O)NCC(C)(C)C(=O)N)O)N
CCOC1=CC=C(C=C1)CC2=C(C=CC(=C2)C3C(C(C(C(O3)CO)O)O)O)Cl
CCC=CCC=CCC=CCC=CCC=CCCCC(=O)O
CCCCC1=NC(=C(N1CC2=CC=C(C=C2)C3=CC=CC=C3C4=NN=N[N-]4)CO)Cl.[K+]
CCCCC(=O)N(CC1=CC=C(C=C1)C2=CC=CC=C2C3=NNN=N3)C(C(C)C)C(=O)O
C1CN=C(N1)NC2=C(C=CC3=NSN=C32)Cl
C1=CC(=CN=C1)C(=O)OC2C(C(C(C(C2OC(=O)C3=CN=CC=C3)OC(=O)C4=CN=CC=C4)OC(=O)C5=CN=CC=C5)OC(=O)C6=CN=CC=C6)OC(=O)C7=CN=CC=C7
CCOC1=NC2=CC=CC(=C2N1CC3=CC=C(C=C3)C4=CC=CC=C4C5=NOC(=O)N5)C(=O)OCC6=C(OC(=O)O6)C
CC(C)NCC(COC1=CC=C(C=C1)CC(=O)N)O
CNC1(CCCCC1=O)C2=CC=CC=C2Cl
CN(C1=C(C(=C(C(=C1I)C(=O)NCC(CO)O)I)C(=O)NCC(CO)O)I)C(=O)CO
COC(=O)C(C1CCCCN1C(=O)OC[N+]2=CC=CC(=C2)C(=O)NC(CO)C(=O)[O-])C3=CC=CC=C3
CC1=C(C(=CC=C1)C)NC(=O)CN2CCN(CC2)CC(COC3=CC=CC=C3OC)O
CCCCCCCN(CC)CCCC(C1=CC=C(C=C1)NS(=O)(=O)C)O
CC(CC1=CC=CC=C1)NC
CN(C)CCOC1=CC=C(C=C1)C(=C(CCCl)C2=CC=CC=C2)C3=CC=CC=C3
C1=CC=C2C(=C1)C(=O)NC2(C3=CC(=C(C=C3)Cl)S(=O)(=O)N)O
CCOC(=O)C(C)CC(CC1=CC=C(C=C1)C2=CC=CC=C2)NC(=O)CCC(=O)O
CCCSC1=NC(=C2C(=N1)N(N=N2)C3CC(C(C3O)O)OCCO)NC4CC4C5=CC(=C(C=C5)F)F
CC(COC1=CC=CC=C1)N(CCCl)CC2=CC=CC=C2
CC(C)(C)NCC(COC1=NSN=C1N2CCOCC2)O
CC(C)(C)NCC(COC1=CC=CC2=C1CCCC2=O)O
CC(=O)OC1C(SC2=CC=CC=C2N(C1=O)CCN(C)C)C3=CC=C(C=C3)OC
C1(=C(N=C(C(=N1)Cl)N)N)C(=O)N=C(N)N
C1C(C2C(O1)C(CO2)O)O
CC(COC1=CC=CC=C1)NC(C)C(C2=CC=C(C=C2)O)O
C1CNCC(C2=CC(=C(C(=C21)Cl)O)O)C3=CC=C(C=C3)O
C1=CC(=CC=C1C2=C(N=CN=C2OCCOC3=NC=C(C=N3)Br)NS(=O)(=O)N)Br
C[N+](C)(C)CCO
CC1C2CC3=C(C1(CCN2CC=C(C)C)C)C=C(C=C3)O
C1=CC=C2C(=C1)C=CC3=CC=CC=C3N2C(=O)N
CC(CC1=CC=CC=C1)NC(=O)C(CCCCN)N
CN1C(CNC2=C1C(=O)NC(=N2)N)CNC3=CC=C(C=C3)C(=O)NC(CCC(=O)O)C(=O)O
C1=CC=C(C=C1)C2=NC3=C(N=C(N=C3N=C2N)N)N
CN1CCC(CC1)C(=O)C2=NC(=CC=C2)NC(=O)C3=C(C=C(C=C3F)F)F
CC1=CN=C(C(=C1OC)C)CS(=O)C2=NC3=C(N2)C=C(C=C3)OC
CC(C)C1(C(=O)N2C(C(=O)N3CCCC3C2(O1)O)CC4=CC=CC=C4)NC(=O)C5CC6C(CC7=CNC8=CC=CC6=C78)N(C5)C
CC1CCC(C(C1)OC(=O)C2=CC=CC=C2O)C(C)C
CCCCCCCCCC(=O)OC1CCC2C1(CCC3C2CCC4=CC(=O)CCC34)C
CCC(C)(C)C(=O)OC1CC(C=C2C1C(C(C=C2)C)CCC3CC(CC(=O)O3)O)C
COC1=C(C=C2C(=C1)C(=NC(=N2)N3CCN(CC3)C(=O)C4=CC=CO4)N)OC
CCC(=O)OC(C(C)C)OP(=O)(CCCCC1=CC=CC=C1)CC(=O)N2CC(CC2C(=O)O)C3CCCCC3
CCOC(=O)C(CCC1=CC=CC=C1)NC2CCC3=CC=CC=C3N(C2=O)CC(=O)O
C1NC2=C(C=C(C(=C2)C(F)(F)F)S(=O)(=O)N)S(=O)(=O)N1
C1C(C2C(O1)C(CO2)O[N+](=O)[O-])O[N+](=O)[O-]
CC1=CC(=NC(=N1)OC(C(=O)O)C(C2=CC=CC=C2)(C3=CC=CC=C3)OC)C
CC(CC1=CC=C(C=C1)O)(C(=O)O)N
CCCCCCCCCCCCCCCC(=O)NC(CCC(=O)NCCCCC(C(=O)NC(CCC(=O)O)C(=O)NC(CC1=CC=CC=C1)C(=O)NC(C(C)CC)C(=O)NC(C)C(=O)NC(CC2=CNC3=CC=CC=C32)C(=O)NC(CC(C)C)C(=O)NC(C(C)C)C(=O)NC(CCCNC(=N)N)C(=O)NCC(=O)NC(CCCNC(=N)N)C(=O)NCC(=O)O)NC(=O)C(C)NC(=O)C(C)NC(=O)C(CCC(=O)N)NC(=O)CNC(=O)C(CCC(=O)O)NC(=O)C(CC(C)C)NC(=O)C(CC4=CC=C(C=C4)O)NC(=O)C(CO)NC(=O)C(CO)NC(=O)C(C(C)C)NC(=O)C(CC(=O)O)NC(=O)C(CO)NC(=O)C(C(C)O)NC(=O)C(CC5=CC=CC=C5)NC(=O)C(C(C)O)NC(=O)CNC(=O)C(CCC(=O)O)NC(=O)C(C)NC(=O)C(CC6=CN=CN6)N)C(=O)O
CCCCCCCCCCNCC=C.C[N+](C)(C)CCCCCCNCC=C.C=CCN.C1C(O1)CCl.Cl.[Cl-]
C1CN(CC1O)C2=C(C=C(C=N2)C(=O)NC3=CC=C(C=C3)OC(F)(F)Cl)C4=CC=NN4
CC1=C(C(=O)C(=C(C1=O)OC)OC)CC=C(C)CCC=C(C)CCC=C(C)CCC=C(C)CCC=C(C)CCC=C(C)CCC=C(C)CCC=C(C)CCC=C(C)C
C1=CC=C(C=C1)CC2NC3=C(C=C(C(=C3)C(F)(F)F)S(=O)(=O)N)S(=O)(=O)N2
CC(CS)C(=O)N1CCCC1C(=O)O
CC(CN1C2=CC=CC=C2SC3=CC=CC=C31)N(C)C
CCN(CCCC(C)NC1=C2C=CC(=CC2=NC=C1)Cl)CCO
CC(C)COC1=CC=C(C=C1)CNC(=O)N(CC2=CC=C(C=C2)F)C3CCN(CC3)C
CCCCCCCCC(=O)NCC1=CC(=C(C=C1)O)OC
CC1=C(C2=C(CCC(O2)(C)CCCC(C)CCCC(C)CCCC(C)C)C(=C1O)C)C
CC(CC1=CC2=C(C(=C1)C(=O)N)N(CC2)CCCO)NCCOC3=CC=CC=C3OCC(F)(F)F
CC(C)(C#N)C1=CC(=CC(=C1)CN2C=NC=N2)C(C)(C)C#N
C1CC(=O)NC(=O)C1N2CC3=C(C2=O)C=CC=C3N
CC(C)(C)NCC(COC1=CC=CC2=C1CCC(=O)N2)O
CC(C)(C)C1=CC=C(C=C1)S(=O)(=O)NC2=C(C(=NC(=N2)C3=NC=CC=N3)OCCO)OC4=CC=CC=C4OC
CCOC(=O)C(CCC1=CC=CC=C1)NC(C)C(=O)N2CCCC2C(=O)O
COC1=C(C(=NC=C1)CS(=O)C2=NC3=C(N2)C=C(C=C3)OC(F)F)OC
C1=C(C=C(C(=C1I)OC2=CC(=C(C(=C2)I)O)I)I)CC(C(=O)O)N
CCOC1=NC2=CC=CC(=C2N1CC3=CC=C(C=C3)C4=CC=CC=C4C5=NNN=N5)C(=O)OC(C)OC(=O)OC6CCCCC6
CCCCN(CCCC)C(=S)[S-].CCCCN(CCCC)C(=S)[S-].[Zn+2]
CC12CCC3C(C1CCC2=O)CCC4=C3C=CC(=C4)OS(=O)(=O)[O-].[Na+]
CCCCN1CCCCC1C(=O)NC2=C(C=CC=C2C)C
CC(=O)[O-].CC(=O)[O-].[Zn+2]
CC#CCC(C)C(C=CC1C(CC2C1CC(=CCCCC(=O)O)C2)O)O
COCCOC(=O)NCCCCC(CCN1CCCC1C(=O)O)NC(=O)OCCOC
CC(C)(C)NCC(COC1=NSN=C1N2CCOCC2)O
COC1=C(C=C2C(=C1)C(=NC(=N2)N3CCN(CC3)C(=O)C4COC5=CC=CC=C5O4)N)OC
CC1(C2CCC1(C(=O)C2)C)C
CCC1C(COC1=O)CC2=CN=CN2C
C1=COC(=C1)CNC2=CC(=C(C=C2C(=O)O)S(=O)(=O)N)Cl
COC1=C(C=C(C=C1)CNC2=NC(=NC=C2C(=O)NCC3=NC=CC=N3)N4CCCC4CO)Cl
CC(CN1CC(=O)NC(=O)C1)N2CC(=O)NC(=O)C2
C1CC(N(C1)C(=O)C2CSSCC(C(=O)NC(C(=O)NC(C(=O)NC(C(=O)NC(C(=O)N2)CC(=O)N)CCC(=O)N)CC3=CC=CC=C3)CC4=CC=C(C=C4)O)N)C(=O)NC(CCCCN)C(=O)NCC(=O)N.C1CC(N(C1)C(=O)C2CSSCC(C(=O)NC(C(=O)NC(C(=O)NC(C(=O)NC(C(=O)N2)CC(=O)N)CCC(=O)N)CC3=CC=CC=C3)CC4=CC=C(C=C4)O)N)C(=O)NC(CCCN=C(N)N)C(=O)NCC(=O)N
CC1CC2=CC=CC=C2N1NC(=O)C3=CC(=C(C=C3)Cl)S(=O)(=O)N
CC(C(C1=CC=CC=C1)O)NC
C1CN(CCC1CC(=O)N2CCC(CC2)C3C4=C(CCC5=C3N=CC(=C5)Br)C=C(C=C4Br)Cl)C(=O)N
CC(=O)N=C1N(N=C(S1)S(=O)(=O)N)C
CC(=O)OC1=CC=CC=C1C(=O)NC2=NC=C(S2)[N+](=O)[O-]
CC1=CC(=C(C(=C1CC2=NCCN2)C)O)C(C)(C)C
C1CN=C(N1)NC2=C(C3=NC=CN=C3C=C2)Br
CN(C1=C(N=C(N=C1N)C2=NN(C3=C2C=CC=N3)CC4=CC=CC=C4F)N)C(=O)OC
C1=CC(=CN=C1)CC(O)(P(=O)(O)O)P(=O)(O)O
C(CC(C(=O)O)N)CN=C(N)N
CC1=C(C(=C(C2=C1OC(CC2)(C)CCCC(C)CCCC(C)CCCC(C)C)C)OC(=O)C)C
CCN(CC)C(=S)[S-].CCN(CC)C(=S)[S-].[Zn+2]
CCNC1CC(S(=O)(=O)C2=C1C=C(S2)S(=O)(=O)N)C
[O-]S(=O)(=O)[O-].[Zn+2]
COC1C(CC2CN3CCC4=C(C3CC2C1C(=O)OC)NC5=C4C=CC(=C5)OC)OC(=O)C6=CC(=C(C(=C6)OC)OC)OC
CCCCCCC(=O)OC1CCC2C1(CCC3C2CCC4=CC(=O)CCC34C)C
CCCNS(=O)(=O)NC1=C(C(=NC=N1)OCCOC2=NC=C(C=N2)Br)C3=CC=C(C=C3)Br
CC1=CC(=CC(=C1CC2=NCCN2)C)C(C)(C)C
CC1(C2CCC(C1C2)CCOCC[N+]3(CCOCC3)CC4=CC(=C(C=C4Br)OC)OC)C
CC1=C(C2=C(N1C(=O)C3=CC=C(C=C3)Cl)C=CC(=C2)OC)CC(=O)O
CC1=CN(C(=O)NC1=O)C2CC(C(O2)COP(=O)([O-])SC3CC(OC3COP(=O)([O-])SC4CC(OC4COP(=O)([O-])SC5CC(OC5COP(=O)([O-])SC6CC(OC6COP(=O)([O-])SC7CC(OC7COP(=O)([O-])SC8CC(OC8COP(=O)([O-])SC9CC(OC9COP(=O)([O-])SC1C(OC(C1OCCOC)N1C=C(C(=NC1=O)N)C)COP(=O)([O-])SC1C(OC(C1OCCOC)N1C=C(C(=O)NC1=O)C)COP(=O)([O-])SC1C(OC(C1OCCOC)N1C=C(C(=NC1=O)N)C)COP(=O)([O-])SC1C(OC(C1OCCOC)N1C=C(C(=NC1=O)N)C)COP(=O)([O-])SC1C(OC(C1OCCOC)N1C=NC2=C1N=C(NC2=O)N)CO)N1C=NC2=C(N=CN=C21)N)N1C=NC2=C1N=C(NC2=O)N)N1C=C(C(=O)NC1=O)C)N1C=C(C(=NC1=O)N)C)N1C=C(C(=O)NC1=O)C)N1C=NC2=C1N=C(NC2=O)N)N1C=C(C(=NC1=O)N)C)SP(=O)([O-])OCC1C(CC(O1)N1C=C(C(=O)NC1=O)C)SP(=O)([O-])OCC1C(CC(O1)N1C=C(C(=NC1=O)N)C)SP(=O)([O-])OCC1C(C(C(O1)N1C=NC2=C1N=C(NC2=O)N)OCCOC)SP(=O)([O-])OCC1C(C(C(O1)N1C=C(C(=NC1=O)N)C)OCCOC)SP(=O)([O-])OCC1C(C(C(O1)N1C=NC2=C(N=CN=C21)N)OCCOC)SP(=O)([O-])OCC1C(C(C(O1)N1C=C(C(=NC1=O)N)C)OCCOC)SP(=O)([O-])OCC1C(C(C(O1)N1C=C(C(=NC1=O)N)C)OCCOC)O
CCC(=O)N(C1=CC=CC=C1)C2(CCN(CC2)CCC3=CC=CS3)COC
C(CO)N(C1=C(C(=C(C(=C1I)C(=O)NCC(CO)O)I)C(=O)NCC(CO)O)I)C(=O)CO
C[N+](C)(C)CCOC(=O)N
Cl[Zn]Cl
C1CC1CN2CCC34CC(=O)CCC3(C2CC5=C4C(=C(C=C5)C(=O)N)O)O
CC1=CN=C(C(=C1OC)C)CS(=O)C2=NC3=C(N2)C=C(C=C3)OC
CC1C(C(C(C(O1)OCC2C(C(C(C(O2)OC3=CC(=C4C(=C3)OC(=CC4=O)C5=CC(=C(C=C5)OC)O)O)O)O)O)O)O)O
CC1=C(C(=C(C2=C1OC(CC2)(C)CCCC(C)CCCC(C)CCCC(C)C)C)OC(=O)CCC(=O)O)C
C1CC(N(C1)C(=O)C2CSSCCC(=O)NC(C(=O)NC(C(=O)NC(C(=O)NC(C(=O)N2)CC(=O)N)CCC(=O)N)CC3=CC=CC=C3)CC4=CC=C(C=C4)O)C(=O)NC(CCCN=C(N)N)C(=O)NCC(=O)N
C1C2=C(C=CC(=C2Cl)Cl)N=C3N1CC(=O)N3
CC(=O)NC1=NN=C(S1)S(=O)(=O)N
CC(=O)C1(C(=C)CC2C1(CCC3C2CCC4=CC(=O)CCC34)C)OC(=O)C
CC1=C(C(C(=C(N1)C)C(=O)OCCN(C)CC2=CC=CC=C2)C3=CC(=CC=C3)[N+](=O)[O-])C(=O)OC
CN1C2CCC1CC(C2)OC(=O)C(CO)C3=CC=CC=C3
C1=CC(=CC=C1C2C(C(=O)N2C3=CC=C(C=C3)F)CCC(C4=CC=C(C=C4)F)O)O
CCCCC1=C(C2=CC=CC=C2O1)C(=O)C3=CC(=C(C(=C3)I)OCCN(CC)CC)I
C1CCC(CC1)N2C(=NN=N2)CCCCOC3=CC4=C(C=C3)NC(=O)CC4
C1=CC=C2C(=C1)C(=NO2)CS(=O)(=O)N
CN1C2=C(C(=O)N(C1=O)C)NC=N2.CN1C2=C(C(=O)N(C1=O)C)NC=N2.C(CN)N
C1=CC(=CC=C1C(=O)NC(CCC(=O)O)C(=O)O)NCC2=CN=C3C(=N2)C(=O)NC(=N3)N
CN1CCN(CC1)CCCN2C3=CC=CC=C3SC4=C2C=C(C=C4)Cl
C1CN(CC2=C1SC=C2)CC3=CC=CC=C3Cl
C1=CC=C(C=C1)C2(C(=O)N(C(=O)N2)COP(=O)(O)O)C3=CC=CC=C3
CN1C(=O)CN=C(C2=C1C=CC(=C2)Cl)C3=CC=CC=C3
CC12CC(C3C(C1CCC2(C(=O)CO)O)CCC4=CC(=O)C=CC34C)O
CC1=CC(=C(C=C1)C)OCCCC(C)(C)C(=O)O
CC1CC2C3CCC4=CC(=O)C=CC4(C3(C(CC2(C1(C(=O)CO)O)C)O)F)C
CN(C)CC1CCCCC1(C2=CC(=CC=C2)OC)O
CC1CC2C3CCC4=CC(=O)C=CC4(C3(C(CC2(C1(C(=O)CO)O)C)O)F)C
"""

# Save the string content to a file in Colab's temporary storage
csv_filename = "ALLDrugswithSMILES.csv"
with open(csv_filename, "w") as f:
    f.write(csv_content)

# Now load the DataFrame
try:
    df = pd.read_csv(csv_filename)
    print(f"\nLoaded dataframe with {len(df)} rows from {csv_filename}.")
    print("DataFrame head:")
    print(df.head())
except FileNotFoundError:
    print(f"Error: File '{csv_filename}' not found. Make sure it was created or uploaded correctly.")
    # Stop execution if file loading fails
    raise SystemExit("Stopping execution due to missing file.")
except Exception as e:
    print(f"An error occurred while reading the CSV: {e}")
    raise SystemExit("Stopping execution due to CSV reading error.")




Loaded dataframe with 835 rows from ALLDrugswithSMILES.csv.
DataFrame head:
                                              Smiles
0               COCCCCC(=NOCCN)C1=CC=C(C=C1)C(F)(F)F
1                          CCCC(C)(COC(=O)N)COC(=O)N
2               CN(C)CCC=C1C2=CC=CC=C2CCC3=CC=CC=C31
3  [Li+].[Li+].[Li+].C(C(=O)[O-])C(CC(=O)[O-])(C(...
4      CN1CCN(CC1)C2=NC3=C(C=CC(=C3)Cl)NC4=CC=CC=C42


In [5]:
def smiles_to_mol(smiles):
    try:
        # Handle potential non-string inputs or common placeholders
        if not isinstance(smiles, str) or "not found" in smiles.lower() or smiles.strip() == "":
             return None
        mol = Chem.MolFromSmiles(smiles)
        # Basic sanitization check (optional but good practice)
        if mol:
             Chem.SanitizeMol(mol)
        return mol # Returns None if Chem.MolFromSmiles fails
    except Exception as e:
        # print(f"Error processing SMILES '{smiles}': {e}") # Uncomment for debugging
        return None

# Apply the conversion
print("\nConverting SMILES to RDKit Mol objects...")
df['mol'] = df['Smiles'].apply(smiles_to_mol)

# Check for and remove rows with invalid SMILES
original_count = len(df)
df.dropna(subset=['mol'], inplace=True)
valid_count = len(df)
invalid_count = original_count - valid_count

if invalid_count > 0:
    print(f"Warning: Removed {invalid_count} rows due to invalid or unparsable SMILES.")
print(f"Processing {valid_count} valid molecules.")


Converting SMILES to RDKit Mol objects...
Processing 833 valid molecules.


[17:12:59] SMILES Parse Error: extra open parentheses while parsing: COC1C(C(C(C(O1)COS(=O)(=O)O)OC2C(C(C(C(O2)C(=O)O)OC3C(C(C(C(O3)COS(=O)(=O)O)OC4C(C(C(C(O4)C(=O)O)OC5C(C(C(C(O5)COS(=O)(=O)O)O)O)NS(=O)(=O)O)O)OS(=O)(=O)O)NS(=O)(=O)O)O)OS(=O)(=O)O)O)NS(=O)(=O)O
[17:12:59] SMILES Parse Error: check for mistakes around position 6:
[17:12:59] COC1C(C(C(C(O1)COS(=O)(=O)O)OC2C(C(C(C(O2
[17:12:59] ~~~~~^
[17:12:59] SMILES Parse Error: Failed parsing SMILES 'COC1C(C(C(C(O1)COS(=O)(=O)O)OC2C(C(C(C(O2)C(=O)O)OC3C(C(C(C(O3)COS(=O)(=O)O)OC4C(C(C(C(O4)C(=O)O)OC5C(C(C(C(O5)COS(=O)(=O)O)O)O)NS(=O)(=O)O)O)OS(=O)(=O)O)NS(=O)(=O)O)O)OS(=O)(=O)O)O)NS(=O)(=O)O' for input: 'COC1C(C(C(C(O1)COS(=O)(=O)O)OC2C(C(C(C(O2)C(=O)O)OC3C(C(C(C(O3)COS(=O)(=O)O)OC4C(C(C(C(O4)C(=O)O)OC5C(C(C(C(O5)COS(=O)(=O)O)O)O)NS(=O)(=O)O)O)OS(=O)(=O)O)NS(=O)(=O)O)O)OS(=O)(=O)O)O)NS(=O)(=O)O'


In [6]:
# 5. RDKit Descriptor Calculation
# ----------------------------------------
print("\nCalculating RDKit descriptors...")

def calculate_rdkit_descriptors(mol):
    """Calculates a selected set of RDKit descriptors."""
    if mol is None:
        # Return a dictionary with None values for consistency
        return {'MolWt': None, 'LogP': None, 'TPSA': None, 'NumHDonors': None,
                'NumHAcceptors': None, 'NumRotatableBonds': None, 'NumRings': None,
                'MolFormula': None}
    try:
        return {
            'MolWt': Descriptors.MolWt(mol),
            'LogP': Crippen.MolLogP(mol), # Using Crippen's LogP calculation
            'TPSA': MolSurf.TPSA(mol),    # Using MolSurf's TPSA calculation
            'NumHDonors': Lipinski.NumHDonors(mol),
            'NumHAcceptors': Lipinski.NumHAcceptors(mol),
            'NumRotatableBonds': Descriptors.NumRotatableBonds(mol),
            'NumRings': Descriptors.RingCount(mol),
            'MolFormula': Descriptors.rdMolDescriptors.CalcMolFormula(mol) # Example: Calculate Mol Formula
        }
    except Exception as e:
        print(f"Error calculating RDKit descriptors for a molecule: {e}")
        # Return None dict in case of calculation error for a valid mol object
        return {'MolWt': None, 'LogP': None, 'TPSA': None, 'NumHDonors': None,
                'NumHAcceptors': None, 'NumRotatableBonds': None, 'NumRings': None,
                'MolFormula': None}

# Apply the function and create a new DataFrame
rdkit_desc_list = df['mol'].apply(calculate_rdkit_descriptors).tolist()
df_rdkit = pd.DataFrame(rdkit_desc_list, index=df.index)

print("RDKit descriptors calculated.")
print("RDKit DataFrame head:")
print(df_rdkit.head())


Calculating RDKit descriptors...
RDKit descriptors calculated.
RDKit DataFrame head:
     MolWt     LogP    TPSA  NumHDonors  NumHAcceptors  NumRotatableBonds  \
0  318.339   3.2015   56.84           1              4                  9   
1  218.253   0.9834  104.64           2              4                  6   
2  277.411   4.1686    3.24           0              1                  3   
3  209.922 -14.2406  140.62           1              7                  5   
4  326.831   3.7227   30.87           1              4                  0   

   NumRings    MolFormula  
0         1  C15H21F3N2O2  
1         0     C9H18N2O4  
2         3       C20H23N  
3         0     C6H5Li3O7  
4         4    C18H19ClN4  


In [8]:
# 6. Mordred Descriptor Calculation
# ----------------------------------------
print("\nCalculating Mordred descriptors (this may take some time)...")

# Initialize Mordred calculator
# ignore_3D=True is crucial when working from SMILES without generating 3D coordinates
mordred_calc = Calculator(descriptors, ignore_3D=True)

# Mordred can calculate descriptors in bulk, which is faster than row-by-row apply
# Filter out any None molecules just in case (should be handled by dropna already)
valid_mols = df['mol'].dropna().tolist()

if valid_mols:
    # Calculate descriptors for all valid molecules
    # The result is a pandas Series/DataFrame indexed like the input list
    try:
        mordred_results = mordred_calc.pandas(valid_mols, nproc=1) # nproc=1 avoids potential multiprocessing issues in Colab

        # Mordred might return results with mixed types (e.g., errors as objects)
        # Convert errors to NaN and ensure numeric types where possible
        mordred_results = mordred_results.apply(pd.to_numeric, errors='coerce')

        # Set the index of mordred results to match the original DataFrame's valid rows
        mordred_results.index = df.index # Crucial for correct alignment

        # Fill missing values (e.g., for molecules where calculation failed) with NaN
        df_mordred = mordred_results.reindex(df.index)

        print(f"Mordred descriptors calculated ({df_mordred.shape[1]} descriptors).")
        print("Mordred DataFrame head:")
        print(df_mordred.head())

        # Optional: Check for columns that are entirely NaN (might indicate consistent errors)
        all_nan_cols = df_mordred.columns[df_mordred.isna().all()].tolist()
        if all_nan_cols:
             print(f"\nWarning: {len(all_nan_cols)} Mordred columns have only NaN values.")
             # print("All NaN columns:", all_nan_cols) # Uncomment to see the names
             # df_mordred = df_mordred.dropna(axis=1, how='all') # Optionally drop them
             # print("Dropped all-NaN columns.")

    except Exception as e:
        print(f"An error occurred during bulk Mordred calculation: {e}")
        # Create an empty DataFrame as a fallback
        df_mordred = pd.DataFrame(index=df.index)
else:
    print("No valid molecules found for Mordred calculation.")
    df_mordred = pd.DataFrame(index=df.index)




Calculating Mordred descriptors (this may take some time)...


/usr/local/lib/python3.11/dist-packages/numpy/core/fromnumeric.py:88: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)
100%|██████████| 833/833 [04:30<00:00,  3.08it/s]


Mordred descriptors calculated (1613 descriptors).
Mordred DataFrame head:
   ABC  ABCGG  nAcid  nBase    SpAbs_A   SpMax_A  SpDiam_A     SpAD_A  \
0  NaN    NaN      0      1  26.793253  2.351839  4.703678  26.793253   
1  NaN    NaN      0      0  16.753257  2.263821  4.527642  16.753257   
2  NaN    NaN      0      1  27.545011  2.447633  4.822978  27.545011   
3  NaN    NaN      3      3        NaN       NaN       NaN        NaN   
4  NaN    NaN      0      3  30.595207  2.466518  4.844876  30.595207   

    SpMAD_A   LogEE_A  ...      SRW10     TSRW10          MW        AMW  \
0  1.217875  3.960814  ...   9.620793  55.012187  318.155513   7.398965   
1  1.116884  3.549234  ...   9.048880  45.566268  218.126657   6.609899   
2  1.311667  3.974821  ...   9.873389  62.814355  277.183050   6.299615   
3       NaN       NaN  ...   9.212837  46.565338  210.051541  10.002454   
4  1.330226  4.090153  ...  10.116298  65.764926  326.129824   7.764996   

        WPath  WPol  Zagreb1  Zagre

In [9]:
# 7. Combine Results
# ----------------------------------------
print("\nCombining original data with calculated descriptors...")

# Drop the temporary 'mol' column from the original df before concatenating
df_combined = pd.concat([df.drop('mol', axis=1), df_rdkit, df_mordred], axis=1)

print("Final DataFrame shape:", df_combined.shape)
print("Final DataFrame head:")
print(df_combined.head())
print("\nFinal DataFrame info:")
df_combined.info()


Combining original data with calculated descriptors...
Final DataFrame shape: (833, 1622)
Final DataFrame head:
                                              Smiles    MolWt     LogP  \
0               COCCCCC(=NOCCN)C1=CC=C(C=C1)C(F)(F)F  318.339   3.2015   
1                          CCCC(C)(COC(=O)N)COC(=O)N  218.253   0.9834   
2               CN(C)CCC=C1C2=CC=CC=C2CCC3=CC=CC=C31  277.411   4.1686   
3  [Li+].[Li+].[Li+].C(C(=O)[O-])C(CC(=O)[O-])(C(...  209.922 -14.2406   
4      CN1CCN(CC1)C2=NC3=C(C=CC(=C3)Cl)NC4=CC=CC=C42  326.831   3.7227   

     TPSA  NumHDonors  NumHAcceptors  NumRotatableBonds  NumRings  \
0   56.84           1              4                  9         1   
1  104.64           2              4                  6         0   
2    3.24           0              1                  3         3   
3  140.62           1              7                  5         0   
4   30.87           1              4                  0         4   

     MolFormula  ABC  ...  

In [12]:
# ----------------------------------------
# output_filename = "ALLDrugswithSMILES_and_Descriptors.csv"
# df_combined.to_csv(output_filename, index=False)
# print(f"\nResults saved to {output_filename}")

In [13]:
from google.colab import files
files.download(output_filename) # Offer download link in Colab

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>